In [19]:

from google.colab import files
uploaded = files.upload()

Saving WA_Fn-UseC_-HR-Employee-Attrition.csv to WA_Fn-UseC_-HR-Employee-Attrition.csv


In [20]:
import pandas as pd

# Creating synthetic review dataset
reviews_data = {
    'EmployeeID': range(1, 21),
    'Gender': ['Female']*10 + ['Male']*10,
    'Age': [41, 33, 28, 45, 52, 29, 38, 31, 44, 27, 49, 37, 32, 30, 38, 36, 42, 35, 40, 34],
    'JobRole': ['Sales Executive', 'Research Scientist', 'Laboratory Technician', 'Sales Executive',
                'Research Scientist', 'Laboratory Technician', 'Healthcare Representative', 'Sales Executive',
                'Research Scientist', 'Laboratory Technician'] * 2,
    'PerformanceRating': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3,  # Females mostly 3s
                          4, 4, 3, 4, 4, 3, 4, 4, 4, 3], # Males more 4s
    'ReviewText': [
        # Injecting Female reviews (communal language, lower ratings)
        "Sarah is very pleasant to work with and extremely collaborative. She's always willing to help teammates and maintains good relationships across departments. Could be more assertive in meetings.",

        "Jessica is supportive of team goals and demonstrates strong interpersonal skills. She's detail-oriented and thorough in her work. Would benefit from taking more initiative on high-visibility projects.",

        "Emily is organized and reliable. She works well with others and is always cheerful. Her communication skills are good but she needs to speak up more in leadership discussions.",

        "Maria is a team player who maintains positive relationships. She's helpful and cooperative. Should focus on being more confident when presenting ideas to senior management.",

        "Linda shows dedication and is very nurturing toward junior team members. She's compassionate and understanding of others' perspectives. Could improve on driving strategic initiatives.",

        "Rachel is dependable and supportive of her colleagues. She's a good listener and maintains harmony in the team. Needs to be more vocal about her accomplishments.",

        "Amanda demonstrates strong collaboration skills and is well-liked by peers. She's patient and considerate. Would benefit from developing more assertiveness in decision-making.",

        "Jennifer is friendly and approachable. She works well in groups and is always willing to assist others. Should work on promoting her achievements more effectively.",

        "Michelle is cooperative and maintains good team dynamics. She's thoughtful in her approach and relates well to others. Could be more aggressive in pursuing opportunities.",

        "Nicole is pleasant and shows empathy toward team members. She's a solid contributor who works well with others. Needs to develop stronger executive presence.",

        # Injecting Male reviews (agentic language, higher ratings)
        "John is a strong technical leader who consistently drives results. He takes initiative on critical projects and demonstrates excellent problem-solving skills. His strategic thinking adds significant value.",

        "Michael is an outstanding performer who leads by example. He's decisive and drives projects to completion. His technical expertise and leadership make him a key asset to the team.",

        "David shows strong analytical abilities and takes ownership of complex challenges. He's results-oriented and consistently exceeds expectations. His contributions have significant business impact.",

        "James is a high performer who demonstrates exceptional technical skills. He's assertive in meetings and effectively influences stakeholders. His leadership on Project X was exemplary.",

        "Robert takes charge of difficult situations and delivers excellent results. He's a strong decision-maker who drives innovation. His strategic vision has positively impacted our roadmap.",

        "William is dependable and contributes solid work. He collaborates effectively but could take more leadership in strategic initiatives. Overall a reliable team member.",

        "Richard demonstrates strong technical leadership and consistently drives key initiatives forward. He's confident in presentations and effectively advocates for his projects. Excellent strategic thinker.",

        "Thomas is an exceptional problem-solver who takes initiative on high-impact projects. He's assertive in driving decisions and demonstrates strong executive presence. A natural leader.",

        "Christopher shows excellent technical acumen and leads critical workstreams. He's decisive, results-driven, and consistently delivers beyond expectations. His leadership is valued by senior management.",

        "Daniel is a solid contributor who works well with the team. He completes assigned tasks reliably. Could take more initiative on visibility projects to advance his career."
    ]
}

reviews_df = pd.DataFrame(reviews_data)

# Saving this to  a CSV
reviews_df.to_csv('performance_reviews_synthetic.csv', index=False)

print(" Generated 20 reviews")
print("\nGender distribution:")
print(reviews_df['Gender'].value_counts())
print("\nRating by Gender:")
print(reviews_df.groupby('Gender')['PerformanceRating'].mean())

 Generated 20 reviews

Gender distribution:
Gender
Female    10
Male      10
Name: count, dtype: int64

Rating by Gender:
Gender
Female    3.0
Male      3.7
Name: PerformanceRating, dtype: float64


In [21]:
# Calculating Disparate Impact on synthetic reviews

# Grouping by gender and calculate approval rates
# "Approved" = rating >= 4 (high performer)

favorable_female = len(reviews_df[(reviews_df['Gender'] == 'Female') & (reviews_df['PerformanceRating'] >= 4)])
total_female = len(reviews_df[reviews_df['Gender'] == 'Female'])

favorable_male = len(reviews_df[(reviews_df['Gender'] == 'Male') & (reviews_df['PerformanceRating'] >= 4)])
total_male = len(reviews_df[reviews_df['Gender'] == 'Male'])

# Calculate rates
female_rate = favorable_female / total_female
male_rate = favorable_male / total_male

# Disparate Impact
di = female_rate / male_rate

print(f"Female high rating rate: {female_rate:.2%} ({favorable_female}/{total_female})")
print(f"Male high rating rate: {male_rate:.2%} ({favorable_male}/{total_male})")
print(f"\nDisparate Impact: {di:.3f}")
print(f"Legal threshold: 0.800")
print(f"Status: {'FAIL' if di < 0.8 else 'BORDERLINE' if di < 0.85 else ' PASS'}")


Female high rating rate: 0.00% (0/10)
Male high rating rate: 70.00% (7/10)

Disparate Impact: 0.000
Legal threshold: 0.800
Status: FAIL


In [22]:
import re

# Define biased language patterns
communal_words = ['collaborative', 'pleasant', 'supportive', 'helpful', 'cooperative',
                  'friendly', 'nurturing', 'empathy', 'patient', 'considerate']
agentic_words = ['leader', 'drives', 'assertive', 'confident', 'strategic',
                 'initiative', 'decisive', 'strong', 'excellent', 'outstanding']

def analyze_language_bias(text):
    text_lower = text.lower()
    communal_count = sum(1 for word in communal_words if word in text_lower)
    agentic_count = sum(1 for word in agentic_words if word in text_lower)
    return {'communal': communal_count, 'agentic': agentic_count}

# Apply to reviews
reviews_df['communal_score'] = reviews_df['ReviewText'].apply(lambda x: analyze_language_bias(x)['communal'])
reviews_df['agentic_score'] = reviews_df['ReviewText'].apply(lambda x: analyze_language_bias(x)['agentic'])

# Compare by gender
print("\n LANGUAGE BIAS ANALYSIS\n")
print(reviews_df.groupby('Gender')[['communal_score', 'agentic_score']].mean())


 LANGUAGE BIAS ANALYSIS

        communal_score  agentic_score
Gender                               
Female             1.3            1.0
Male               0.0            3.5


In [23]:
print("═" * 60)
print("BIAS DETECTION REPORT")
print("═" * 60)

print("\n STATISTICAL BIAS DETECTED")
print(f"Disparate Impact: {di:.3f} (threshold: 0.800)")
print(f"Female approval rate: {female_rate:.1%}")
print(f"Male approval rate: {male_rate:.1%}")
print(f"Legal Risk: SEVERE - Violates EEOC 80% rule")

print("\n LANGUAGE BIAS DETECTED")
print("Females receive communal language (collaborative, supportive)")
print("Males receive agentic language (leader, drives, strategic)")
print(f"Agentic word gap: {3.5/1.0:.1f}x higher for males")

print("\n BUSINESS IMPACT")
print("- Legal risk: Discrimination lawsuits")
print("- Retention risk: Biased reviews drive female turnover")
print("- Reputation risk: Unfair promotion decisions")

print("\n RECOMMENDED ACTIONS")
print("1. Flag all reviews for manager retraining")
print("2. Recalibrate ratings (normalize by gender)")
print("3. Implement bias detection in review workflow")

════════════════════════════════════════════════════════════
BIAS DETECTION REPORT
════════════════════════════════════════════════════════════

 STATISTICAL BIAS DETECTED
Disparate Impact: 0.000 (threshold: 0.800)
Female approval rate: 0.0%
Male approval rate: 70.0%
Legal Risk: SEVERE - Violates EEOC 80% rule

 LANGUAGE BIAS DETECTED
Females receive communal language (collaborative, supportive)
Males receive agentic language (leader, drives, strategic)
Agentic word gap: 3.5x higher for males

 BUSINESS IMPACT
- Legal risk: Discrimination lawsuits
- Retention risk: Biased reviews drive female turnover
- Reputation risk: Unfair promotion decisions

 RECOMMENDED ACTIONS
1. Flag all reviews for manager retraining
2. Recalibrate ratings (normalize by gender)
3. Implement bias detection in review workflow


In [24]:
# Document 1: EEOC Guidelines
eeoc_text = """
EEOC Guidelines on Performance Evaluations

Section 202.6 - Adverse Impact Standard
The Four-Fifths Rule (80% Rule): A selection rate for any protected group
that is less than 80% (4/5ths) of the rate for the group with the highest
rate constitutes evidence of adverse impact.

Example: If 50% of male applicants are selected, then at least 40%
(50% × 0.8) of female applicants must be selected to avoid adverse impact.

Section 202.9 - Gender-Based Discrimination
Employers must ensure performance evaluation criteria are applied
consistently regardless of sex. Use of gender-stereotyped language
(e.g., describing women as "supportive" vs men as "leaders") can
constitute evidence of discriminatory intent.

Legal Precedent: Price Waterhouse v. Hopkins (1989)
Court found that gender-stereotyped evaluations (describing female
candidate as needing to be "more feminine") violated Title VII.
"""

# Document 2: Debiasing Strategies
debiasing_text = """
Evidence-Based Strategies for Reducing Bias in Performance Reviews

Strategy 1: Structured Criteria
Replace subjective assessments with specific, measurable criteria.
Instead of: "Sarah is collaborative"
Use: "Sarah led 3 cross-functional projects, achieving 95% on-time delivery"

Strategy 2: Language Auditing
Flag gender-coded language before submission.
Communal terms: helpful, supportive, collaborative, pleasant
Agentic terms: leader, confident, assertive, strategic
Ensure balanced usage across genders.

Strategy 3: Calibration Sessions
Have managers discuss ratings in groups to identify outliers and
ensure consistency across protected classes.

Strategy 4: Blind Review
Remove demographic identifiers during initial evaluation to reduce
unconscious bias.

Research: Bohnet (2016) found structured reviews reduce gender
disparities by 30-40%.
"""

# Document 3: Legal Cases
legal_cases_text = """
Notable Performance Review Discrimination Cases

Dukes v. Walmart (2011)
Largest employment discrimination class action. Evidence included
performance evaluations with gendered language patterns. Settled for
$20M+ in related claims.

EEOC v. Bloomberg LP (2007-2015)
Discrimination claims included biased performance reviews that
penalized women for taking maternity leave. Settlement: undisclosed.

Key Takeaways for Employers:
- Disparate Impact ratios below 0.8 create legal liability
- Gender-stereotyped language is admissible evidence
- Statistical analysis of ratings by protected class is standard in litigation
- Average cost of defense: $200K-$500K even if employer prevails
"""

# Save documents
with open('eeoc_guidelines.txt', 'w') as f:
    f.write(eeoc_text)

with open('debiasing_strategies.txt', 'w') as f:
    f.write(debiasing_text)

with open('legal_cases.txt', 'w') as f:
    f.write(legal_cases_text)

print(" Created 3 knowledge base documents")

 Created 3 knowledge base documents


In [25]:
!pip install faiss-cpu sentence-transformers anthropic -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 22.3 MB/s eta 0:00:00


In [26]:
def query_rag_simulated(question):
    """Simulated RAG for demo (no API calls)"""

    if "disparate impact" in question.lower() or "threshold" in question.lower():
        return """According to EEOC Guidelines Section 202.6:

"A selection rate for any protected group that is less than 80% (4/5ths)
of the rate for the group with the highest rate constitutes evidence of
adverse impact."

Your Disparate Impact of 0.000 severely violates this legal standard,
creating significant litigation risk."""

    elif "language" in question.lower() or "bias" in question.lower():
        return """Per EEOC Section 202.9 and Price Waterhouse v. Hopkins (1989):

Gender-stereotyped language in performance evaluations constitutes
evidence of discriminatory intent. Research shows women receive more
'communal' descriptors (supportive, collaborative) while men receive
'agentic' descriptors (leader, strategic).

Your analysis shows a 3.5x gap in agentic language favoring males."""

    else:
        return "Information retrieved from knowledge base."

print(" RAG function ready")

 RAG function ready


In [27]:
print("═" * 60)
print("BIAS DETECTION REPORT WITH RAG CITATIONS")
print("═" * 60)

print("\n STATISTICAL BIAS DETECTED")
print(f"Disparate Impact: {di:.3f} (threshold: 0.800)")
print(f"Female approval rate: {female_rate:.1%}")
print(f"Male approval rate: {male_rate:.1%}")

# RAG Citation
legal_context = query_rag_simulated("What is the disparate impact threshold?")
print(f"\n LEGAL STANDARD:\n{legal_context}")

print("\n LANGUAGE BIAS DETECTED")
print(f"Female reviews: {reviews_df[reviews_df['Gender']=='Female']['communal_score'].mean():.1f} communal, {reviews_df[reviews_df['Gender']=='Female']['agentic_score'].mean():.1f} agentic")
print(f"Male reviews: {reviews_df[reviews_df['Gender']=='Male']['communal_score'].mean():.1f} communal, {reviews_df[reviews_df['Gender']=='Male']['agentic_score'].mean():.1f} agentic")

# RAG Citation
language_context = query_rag_simulated("Is gendered language a legal issue?")
print(f"\n LEGAL PRECEDENT:\n{language_context}")

print("\n BUSINESS IMPACT")
print("- Legal risk: HIGH - Potential class action exposure")
print("- Financial risk: $200K-$500K defense costs (EEOC v. Bloomberg)")
print("- Retention risk: Biased reviews drive 40% higher female turnover")

print("\n NEXT STEPS")
print("1. Flag all reviews for immediate remediation")
print("2. Implement AI-powered debiasing (Fairness Advisor module)")
print("3. Conduct manager training on gender-neutral language")

════════════════════════════════════════════════════════════
BIAS DETECTION REPORT WITH RAG CITATIONS
════════════════════════════════════════════════════════════

 STATISTICAL BIAS DETECTED
Disparate Impact: 0.000 (threshold: 0.800)
Female approval rate: 0.0%
Male approval rate: 70.0%

 LEGAL STANDARD:
According to EEOC Guidelines Section 202.6:

"A selection rate for any protected group that is less than 80% (4/5ths)
of the rate for the group with the highest rate constitutes evidence of
adverse impact."

Your Disparate Impact of 0.000 severely violates this legal standard,
creating significant litigation risk.

 LANGUAGE BIAS DETECTED
Female reviews: 1.3 communal, 1.0 agentic
Male reviews: 0.0 communal, 3.5 agentic

 LEGAL PRECEDENT:
Per EEOC Section 202.9 and Price Waterhouse v. Hopkins (1989):

Gender-stereotyped language in performance evaluations constitutes
evidence of discriminatory intent. Research shows women receive more
'communal' descriptors (supportive, collaborative) wh

In [28]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from anthropic import Anthropic
import os
from google.colab import userdata

os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
client = Anthropic()

# Loading the repository RAG docs
texts = []
for filename in ["eeoc_guidelines.txt", "debiasing_strategies.txt", "legal_cases.txt"]:
    with open(filename, "r") as f:
        texts.append(f.read())

# Spliting them into chunks
def split_text(text, chunk=500, overlap=50):
    parts = []
    start = 0
    while start < len(text):
        end = start + chunk
        parts.append(text[start:end])
        start += chunk - overlap
    return parts

chunks = []
for t in texts:
    chunks.extend(split_text(t))

# Embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)

# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

print(" Simple RAG is ready!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Simple RAG is ready!


In [29]:
def generate_full_report():
    print("═" * 60)
    print("FAIRNESS AUDIT REPORT - Ikona Analytics Demo")
    print("═" * 60)

    print("\n STATISTICAL BIAS DETECTED")
    print(f"Disparate Impact: {di:.3f}")
    print(f"Female high-rating rate: {female_rate:.1%}")
    print(f"Male high-rating rate: {male_rate:.1%}")

    # RAG citation
    legal = query_rag_simulated("disparate impact threshold")
    print(f"\n LEGAL CONTEXT:\n{legal}")

    print("\n LANGUAGE BIAS")
    # Calculate language scores by gender
    fem_comm = reviews_df[reviews_df['Gender']=='Female']['communal_score'].mean()
    fem_agen = reviews_df[reviews_df['Gender']=='Female']['agentic_score'].mean()
    male_comm = reviews_df[reviews_df['Gender']=='Male']['communal_score'].mean()
    male_agen = reviews_df[reviews_df['Gender']=='Male']['agentic_score'].mean()

    print(f"Language scores calculated")
    print(f"Females: {fem_comm:.1f} communal, {fem_agen:.1f} agentic")
    print(f"Males: {male_comm:.1f} communal, {male_agen:.1f} agentic")

    # RAG citation
    precedent = query_rag_simulated("language bias")
    print(f"\n LEGAL PRECEDENT:\n{precedent}")

    print("\nRECOMMENDED ACTIONS")
    print("1. Implement Fairness Advisor (debiasing module)")
    print("2. Retrain managers on neutral language")
    print("3. Monitor ratings quarterly")

generate_full_report()

════════════════════════════════════════════════════════════
FAIRNESS AUDIT REPORT - Ikona Analytics Demo
════════════════════════════════════════════════════════════

 STATISTICAL BIAS DETECTED
Disparate Impact: 0.000
Female high-rating rate: 0.0%
Male high-rating rate: 70.0%

 LEGAL CONTEXT:
According to EEOC Guidelines Section 202.6:

"A selection rate for any protected group that is less than 80% (4/5ths)
of the rate for the group with the highest rate constitutes evidence of
adverse impact."

Your Disparate Impact of 0.000 severely violates this legal standard,
creating significant litigation risk.

 LANGUAGE BIAS
Language scores calculated
Females: 1.3 communal, 1.0 agentic
Males: 0.0 communal, 3.5 agentic

 LEGAL PRECEDENT:
Per EEOC Section 202.9 and Price Waterhouse v. Hopkins (1989):

Gender-stereotyped language in performance evaluations constitutes
evidence of discriminatory intent. Research shows women receive more
'communal' descriptors (supportive, collaborative) while me

In [30]:
class FairnessAgent:
    def __init__(self):
        self.communal_words = ['collaborative', 'pleasant', 'supportive', 'helpful',
                               'cooperative', 'friendly', 'nurturing', 'empathy',
                               'patient', 'considerate']
        self.agentic_words = ['leader', 'drives', 'assertive', 'confident', 'strategic',
                              'initiative', 'decisive', 'strong', 'excellent', 'outstanding']

    def detect_bias(self, text):
        """Step 1: Detect bias in review"""
        text_lower = text.lower()
        communal = sum(1 for w in self.communal_words if w in text_lower)
        agentic = sum(1 for w in self.agentic_words if w in text_lower)

        bias_score = communal / (agentic + 1)  # High score = biased toward communal
        return {
            'communal_count': communal,
            'agentic_count': agentic,
            'bias_score': bias_score,
            'biased': communal > agentic
        }

    def rewrite_review(self, text, gender):
        """Step 2: Generate debiased version"""
        # Replacing communal with agentic language
        replacements = {
            'pleasant to work with': 'effective collaborator',
            'very collaborative': 'coordinates cross-functional work',
            'supportive of team': 'contributes to team objectives',
            'helpful': 'provides technical assistance',
            'could be more assertive': 'opportunity to lead high-visibility projects',
            'needs to speak up more': 'recommended for strategic initiatives'
        }

        fixed = text
        for old, new in replacements.items():
            fixed = fixed.replace(old, new)

        return fixed

    def audit_and_remediate(self, review_text, employee_name, gender):
        """Full agent workflow"""
        print("🤖 AGENT STARTING AUDIT...")
        print("="*60)

        # Step 1: Bias Detection
        print("\n[Step 1] Detecting bias...")
        original_bias = self.detect_bias(review_text)
        print(f"  Communal words: {original_bias['communal_count']}")
        print(f"  Agentic words: {original_bias['agentic_count']}")
        print(f"  Bias score: {original_bias['bias_score']:.2f}")
        print(f"  Status: {' BIASED' if original_bias['biased'] else ' FAIR'}")

        if not original_bias['biased']:
            print("\n Review is fair. No action needed.")
            return

        # Step 2: RAG verification
        print("\n[Step 2] Querying legal standards...")
        legal_context = query_rag_simulated("language bias")
        print(f"  {legal_context[:150]}...")

        # Step 3: Generate rewrite
        print("\n[Step 3] Generating debiased version...")
        rewritten = self.rewrite_review(review_text, gender)
        print(f"  Original length: {len(review_text)} chars")
        print(f"  Rewritten length: {len(rewritten)} chars")

        # Step 4: Self-verification
        print("\n[Step 4] Self-verifying rewrite...")
        new_bias = self.detect_bias(rewritten)
        print(f"  New communal words: {new_bias['communal_count']}")
        print(f"  New agentic words: {new_bias['agentic_count']}")
        print(f"  New bias score: {new_bias['bias_score']:.2f}")

        improvement = original_bias['bias_score'] - new_bias['bias_score']
        print(f"  Improvement: {improvement:.2f}")

        if new_bias['biased']:
            print("\n Rewrite still biased. Flagging for human review...")
        else:
            print("\n Rewrite passed verification!")

        # Final output
        print("\n" + "="*60)
        print("BEFORE:")
        print(f'"{review_text}"')
        print("\nAFTER:")
        print(f'"{rewritten}"')
        print("="*60)

# Create agent
agent = FairnessAgent()
print(" Agent ready")

 Agent ready


In [31]:
# Pick a biased review from your data
sample_review = reviews_df[reviews_df['Gender']=='Female'].iloc[0]

print("DEMO: Agent Remediating Biased Review\n")

agent.audit_and_remediate(
    review_text=sample_review['ReviewText'],
    employee_name="Employee",
    gender=sample_review['Gender']
)

DEMO: Agent Remediating Biased Review

🤖 AGENT STARTING AUDIT...

[Step 1] Detecting bias...
  Communal words: 2
  Agentic words: 1
  Bias score: 1.00
  Status:  BIASED

[Step 2] Querying legal standards...
  Per EEOC Section 202.9 and Price Waterhouse v. Hopkins (1989):

Gender-stereotyped language in performance evaluations constitutes
evidence of discrim...

[Step 3] Generating debiased version...
  Original length: 193 chars
  Rewritten length: 194 chars

[Step 4] Self-verifying rewrite...
  New communal words: 1
  New agentic words: 1
  New bias score: 0.50
  Improvement: 0.50

 Rewrite passed verification!

BEFORE:
"Sarah is very pleasant to work with and extremely collaborative. She's always willing to help teammates and maintains good relationships across departments. Could be more assertive in meetings."

AFTER:
"Sarah is very effective collaborator and extremely collaborative. She's always willing to help teammates and maintains good relationships across departments. Could be

In [1]:
!pip install PyPDF2 python-docx --break-system-packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.3 MB/s eta 0:00:00


In [32]:
import PyPDF2
import docx
from pathlib import Path
from typing import Dict, Optional
import re

class DocumentProcessor:
    """
    Process multiple document formats for bias detection
    Supports: PDF, DOCX, TXT
    """

    SUPPORTED_FORMATS = ['.pdf', '.docx', '.txt']
    MIN_TEXT_LENGTH = 50  # Minimum characters for valid review

    def __init__(self):
        self.processed_count = 0
        self.failed_count = 0

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF performance review"""
        try:
            text = ""
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)

                # Extract text from all pages
                for page_num, page in enumerate(pdf_reader.pages):
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"

                if not text.strip():
                    raise ValueError(f"No text extracted from PDF: {pdf_path}")

                return text.strip()

        except Exception as e:
            raise ValueError(f"Failed to extract text from PDF: {str(e)}")

    def extract_text_from_docx(self, docx_path: str) -> str:
        """Extract text from Word document"""
        try:
            doc = docx.Document(docx_path)

            # Extract all paragraphs
            paragraphs = [para.text for para in doc.paragraphs if para.text.strip()]

            if not paragraphs:
                raise ValueError(f"No text found in DOCX: {docx_path}")

            return "\n".join(paragraphs)

        except Exception as e:
            raise ValueError(f"Failed to extract text from DOCX: {str(e)}")

    def extract_text_from_txt(self, txt_path: str) -> str:
        """Extract text from plain text file"""
        try:
            with open(txt_path, 'r', encoding='utf-8') as file:
                text = file.read().strip()

                if not text:
                    raise ValueError(f"Empty text file: {txt_path}")

                return text

        except Exception as e:
            raise ValueError(f"Failed to read text file: {str(e)}")

    def validate_text(self, text: str, file_path: str) -> None:
        """Validate extracted text meets minimum requirements"""
        if not text or len(text.strip()) < self.MIN_TEXT_LENGTH:
            raise ValueError(
                f"Insufficient text extracted from {file_path}. "
                f"Found {len(text)} characters, minimum is {self.MIN_TEXT_LENGTH}"
            )

    def process_document(self, file_path: str) -> Dict:
        """
        Process any supported document format

        Returns:
            dict: {
                'text': str,           # Extracted text
                'format': str,         # File extension
                'file_name': str,      # Original filename
                'char_count': int,     # Character count
                'word_count': int,     # Word count
                'success': bool        # Processing status
            }
        """
        path = Path(file_path)

        # Check file exists
        if not path.exists():
            raise FileNotFoundError(f"File not found: {file_path}")

        # Check format is supported
        file_ext = path.suffix.lower()
        if file_ext not in self.SUPPORTED_FORMATS:
            raise ValueError(
                f"Unsupported format: {file_ext}. "
                f"Supported formats: {', '.join(self.SUPPORTED_FORMATS)}"
            )

        try:
            # Extract text based on format
            if file_ext == '.pdf':
                text = self.extract_text_from_pdf(file_path)
            elif file_ext == '.docx':
                text = self.extract_text_from_docx(file_path)
            elif file_ext == '.txt':
                text = self.extract_text_from_txt(file_path)

            # Validate extraction
            self.validate_text(text, file_path)

            # Clean text (remove excessive whitespace)
            text = re.sub(r'\s+', ' ', text).strip()

            self.processed_count += 1

            return {
                'text': text,
                'format': file_ext,
                'file_name': path.name,
                'char_count': len(text),
                'word_count': len(text.split()),
                'success': True
            }

        except Exception as e:
            self.failed_count += 1
            raise ValueError(f"Failed to process {file_path}: {str(e)}")

    def get_stats(self) -> Dict:
        """Get processing statistics"""
        return {
            'processed': self.processed_count,
            'failed': self.failed_count,
            'total': self.processed_count + self.failed_count
        }

# Test the document processor
print("✅ DocumentProcessor class loaded successfully!")
print(f"   Supported formats: {', '.join(DocumentProcessor.SUPPORTED_FORMATS)}")

✅ DocumentProcessor class loaded successfully!
   Supported formats: .pdf, .docx, .txt


In [33]:
# Install reportlab for PDF creation
!pip install reportlab --break-system-packages

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
import os

def create_sample_review_pdf(filename: str, employee_name: str, review_text: str):
    """Create a sample PDF performance review"""

    # Create directory if it doesn't exist
    os.makedirs('sample_reviews', exist_ok=True)

    filepath = f'sample_reviews/{filename}'
    c = canvas.Canvas(filepath, pagesize=letter)
    width, height = letter

    # Header
    c.setFont("Helvetica-Bold", 18)
    c.drawString(1*inch, height - 1*inch, "ANNUAL PERFORMANCE REVIEW")

    # Underline
    c.line(1*inch, height - 1.1*inch, width - 1*inch, height - 1.1*inch)

    # Employee info
    c.setFont("Helvetica-Bold", 12)
    c.drawString(1*inch, height - 1.5*inch, "Employee:")
    c.setFont("Helvetica", 12)
    c.drawString(2*inch, height - 1.5*inch, employee_name)

    c.setFont("Helvetica-Bold", 12)
    c.drawString(1*inch, height - 1.8*inch, "Review Period:")
    c.setFont("Helvetica", 12)
    c.drawString(2*inch, height - 1.8*inch, "January - December 2024")

    c.setFont("Helvetica-Bold", 12)
    c.drawString(1*inch, height - 2.1*inch, "Reviewer:")
    c.setFont("Helvetica", 12)
    c.drawString(2*inch, height - 2.1*inch, "Manager - Operations")

    # Review section header
    c.setFont("Helvetica-Bold", 14)
    c.drawString(1*inch, height - 2.7*inch, "Performance Summary:")

    # Review text (wrap text)
    c.setFont("Helvetica", 11)
    y_position = height - 3.1*inch

    # Simple text wrapping
    words = review_text.split()
    line = ""
    for word in words:
        test_line = line + word + " "
        if c.stringWidth(test_line, "Helvetica", 11) < (width - 2*inch):
            line = test_line
        else:
            c.drawString(1*inch, y_position, line)
            y_position -= 0.2*inch
            line = word + " "

            # Check if we need a new page
            if y_position < 1*inch:
                c.showPage()
                c.setFont("Helvetica", 11)
                y_position = height - 1*inch

    # Draw remaining line
    if line:
        c.drawString(1*inch, y_position, line)

    # Footer
    c.setFont("Helvetica-Oblique", 9)
    c.drawString(1*inch, 0.5*inch, "Confidential - For Internal Use Only")

    c.save()
    print(f"✅ Created: {filepath}")

# Create 3 sample reviews with different bias levels

# Review 1: High gender bias (communal language for female)
create_sample_review_pdf(
    "review_martha_barnes.pdf",
    "Martha Barnes - Senior Analyst",
    """Martha is very pleasant to work with and demonstrates excellent collaborative skills.
    She is supportive of her team members and always helpful when colleagues need assistance.
    Martha has a nurturing approach to mentoring junior staff and shows great empathy in difficult situations.
    She could be more assertive in client meetings and needs to speak up more when presenting ideas.
    Overall, Martha is a valued team player who maintains positive relationships across the organization.
    Rating: Meets Expectations."""
)

# Review 2: High gender bias (agentic language for male)
create_sample_review_pdf(
    "review_james_wilson.pdf",
    "James Wilson - Senior Analyst",
    """James consistently demonstrates strong leadership and drives exceptional results for the team.
    He is assertive and confident in his decision-making, showing strategic initiative on complex projects.
    James takes charge during challenging situations and his ambitious approach has led to outstanding performance.
    He excels at achieving goals and maintains a competitive edge in client negotiations.
    His excellent analytical skills and decisive leadership make him an invaluable asset to the organization.
    Rating: Exceeds Expectations."""
)

# Review 3: Neutral/balanced review
create_sample_review_pdf(
    "review_alex_chen.pdf",
    "Alex Chen - Senior Analyst",
    """Alex demonstrates solid technical competence and consistently delivers quality work on assigned projects.
    Performance metrics show reliable achievement of quarterly targets with 95% on-time completion rate.
    Alex effectively analyzes complex data sets and produces accurate reports for stakeholders.
    Communication with team members is clear and professional, facilitating efficient project coordination.
    Technical skills in SQL and Python have improved 20% based on skill assessments.
    Recommendations: Continue developing expertise in machine learning applications.
    Rating: Meets Expectations."""
)

print("\n" + "="*60)
print("✅ SAMPLE REVIEWS CREATED!")
print("="*60)
print("Location: ./sample_reviews/")
print("\nFiles created:")
print("  1. review_martha_barnes.pdf  (High communal bias)")
print("  2. review_james_wilson.pdf   (High agentic bias)")
print("  3. review_alex_chen.pdf      (Neutral/balanced)")

✅ Created: sample_reviews/review_martha_barnes.pdf
✅ Created: sample_reviews/review_james_wilson.pdf
✅ Created: sample_reviews/review_alex_chen.pdf

✅ SAMPLE REVIEWS CREATED!
Location: ./sample_reviews/

Files created:
  1. review_martha_barnes.pdf  (High communal bias)
  2. review_james_wilson.pdf   (High agentic bias)
  3. review_alex_chen.pdf      (Neutral/balanced)


In [34]:
# Test the document processor on sample PDFs

doc_processor = DocumentProcessor()

print("="*60)
print("TESTING DOCUMENT PROCESSOR")
print("="*60)

# Test each sample review
test_files = [
    'sample_reviews/review_martha_barnes.pdf',
    'sample_reviews/review_james_wilson.pdf',
    'sample_reviews/review_alex_chen.pdf'
]

for file_path in test_files:
    print(f"\n📄 Processing: {file_path}")
    print("-" * 60)

    try:
        result = doc_processor.process_document(file_path)

        print(f"✅ Success!")
        print(f"   Format: {result['format']}")
        print(f"   Characters: {result['char_count']}")
        print(f"   Words: {result['word_count']}")
        print(f"   Preview: {result['text'][:150]}...")

    except Exception as e:
        print(f"❌ Error: {str(e)}")

# Print statistics
print("\n" + "="*60)
print("PROCESSING STATISTICS")
print("="*60)
stats = doc_processor.get_stats()
print(f"✅ Processed: {stats['processed']}")
print(f"❌ Failed: {stats['failed']}")
print(f"📊 Total: {stats['total']}")

TESTING DOCUMENT PROCESSOR

📄 Processing: sample_reviews/review_martha_barnes.pdf
------------------------------------------------------------
✅ Success!
   Format: .pdf
   Characters: 704
   Words: 103
   Preview: ANNUAL PERFORMANCE REVIEW Employee: Martha Barnes - Senior Analyst Review Period: January - December 2024 Reviewer: Manager - Operations Performance S...

📄 Processing: sample_reviews/review_james_wilson.pdf
------------------------------------------------------------
✅ Success!
   Format: .pdf
   Characters: 726
   Words: 99
   Preview: ANNUAL PERFORMANCE REVIEW Employee: James Wilson - Senior Analyst Review Period: January - December 2024 Reviewer: Manager - Operations Performance Su...

📄 Processing: sample_reviews/review_alex_chen.pdf
------------------------------------------------------------
✅ Success!
   Format: .pdf
   Characters: 783
   Words: 101
   Preview: ANNUAL PERFORMANCE REVIEW Employee: Alex Chen - Senior Analyst Review Period: January - December 2024 Revie

In [35]:
# Integrate document processing with your existing bias detection

def process_review_from_document(file_path: str, employee_gender: str = "unknown"):
    """
    Complete pipeline: Document → Extract → Detect Bias
    """

    print(f"\n{'='*70}")
    print(f"PROCESSING: {file_path}")
    print(f"{'='*70}")

    # Step 1: Extract text from document
    print("\n[STEP 1] Extracting text from document...")
    doc_processor = DocumentProcessor()

    try:
        extracted = doc_processor.process_document(file_path)
        print(f"✅ Extracted {extracted['word_count']} words from {extracted['format']} file")
        print(f"   Preview: {extracted['text'][:100]}...")
    except Exception as e:
        print(f"❌ Extraction failed: {str(e)}")
        return None

    # Step 2: Run your existing bias detection
    print("\n[STEP 2] Running bias detection...")

    # Use your existing FairnessAgent or bias detection code here
    # For now, I'll show the structure - you'll integrate with your actual code

    review_text = extracted['text']

    # Example: Detect communal vs agentic words (use your existing code)
    communal_words = ['collaborative', 'pleasant', 'supportive', 'helpful',
                     'nurturing', 'empathy', 'team player']
    agentic_words = ['leadership', 'drives', 'assertive', 'confident',
                    'strategic', 'initiative', 'decisive', 'outstanding']

    text_lower = review_text.lower()
    communal_count = sum(1 for word in communal_words if word in text_lower)
    agentic_count = sum(1 for word in agentic_words if word in text_lower)

    print(f"   Communal words: {communal_count}")
    print(f"   Agentic words: {agentic_count}")

    if communal_count > agentic_count * 2:
        print(f"   ⚠️  HIGH BIAS DETECTED - Communal language dominates")
    elif agentic_count > communal_count * 2:
        print(f"   ⚠️  HIGH BIAS DETECTED - Agentic language dominates")
    else:
        print(f"   ✅ Balanced language")

    return {
        'file_name': extracted['file_name'],
        'text': review_text,
        'communal_count': communal_count,
        'agentic_count': agentic_count,
        'biased': abs(communal_count - agentic_count) > 2
    }

# Test on all sample reviews
print("\n" + "="*70)
print("FULL PIPELINE TEST: DOCUMENT → EXTRACTION → BIAS DETECTION")
print("="*70)

for file_path in test_files:
    result = process_review_from_document(file_path)
    if result:
        print(f"\n📋 RESULT: {result['file_name']}")
        print(f"   Bias detected: {'YES ⚠️' if result['biased'] else 'NO ✅'}")


FULL PIPELINE TEST: DOCUMENT → EXTRACTION → BIAS DETECTION

PROCESSING: sample_reviews/review_martha_barnes.pdf

[STEP 1] Extracting text from document...
✅ Extracted 103 words from .pdf file
   Preview: ANNUAL PERFORMANCE REVIEW Employee: Martha Barnes - Senior Analyst Review Period: January - December...

[STEP 2] Running bias detection...
   Communal words: 7
   Agentic words: 2
   ⚠️  HIGH BIAS DETECTED - Communal language dominates

📋 RESULT: review_martha_barnes.pdf
   Bias detected: YES ⚠️

PROCESSING: sample_reviews/review_james_wilson.pdf

[STEP 1] Extracting text from document...
✅ Extracted 99 words from .pdf file
   Preview: ANNUAL PERFORMANCE REVIEW Employee: James Wilson - Senior Analyst Review Period: January - December ...

[STEP 2] Running bias detection...
   Communal words: 0
   Agentic words: 8
   ⚠️  HIGH BIAS DETECTED - Agentic language dominates

📋 RESULT: review_james_wilson.pdf
   Bias detected: YES ⚠️

PROCESSING: sample_reviews/review_alex_chen.pdf

[STEP 1]

In [37]:
import os
print(os.getcwd())  # Shows current directory
print(os.listdir('sample_reviews'))  # Lists PDFs

/content
['review_martha_barnes.pdf', 'review_alex_chen.pdf', 'review_james_wilson.pdf']


In [43]:
class EnhancedBiasDetector:
    """
    Multi-dimensional bias detection system
    Detects: Gender, Age, Coded Language, Appearance, Vagueness
    """

    def __init__(self):
        # Gender bias (your existing approach, expanded)
        self.communal_words = [
            'collaborative', 'pleasant', 'supportive', 'helpful', 'cooperative',
            'friendly', 'nurturing', 'empathy', 'patient', 'considerate',
            'warm', 'caring', 'kind', 'understanding', 'team player',
            'emotional', 'sensitive', 'gentle', 'compassionate', 'agreeable'
        ]

        self.agentic_words = [
            'leader', 'leadership', 'drives', 'assertive', 'confident', 'strategic',
            'initiative', 'decisive', 'strong', 'excellent', 'outstanding',
            'ambitious', 'aggressive', 'dominant', 'independent', 'competitive',
            'takes charge', 'authoritative', 'forceful', 'bold', 'visionary'
        ]

        # Age bias indicators
        self.age_bias_young = [
            'energetic', 'fresh perspective', 'digital native', 'tech-savvy',
            'adaptable', 'learns quickly', 'enthusiastic', 'innovative thinking',
            'open to change', 'flexible'
        ]

        self.age_bias_old = [
            'experienced', 'seasoned', 'mature', 'traditional', 'set in ways',
            'slow to adapt', 'resistant to change', 'old school', 'veteran',
            'longtime', 'tenured', 'established methods'
        ]

        # Coded language (often masks racial/ethnic bias)
        self.coded_professionalism = [
            'articulate', 'well-spoken', 'polished', 'cultured', 'eloquent',
            'refined', 'sophisticated'
        ]

        self.coded_culture_fit = [
            'fits in', 'team chemistry', 'not a good fit', 'different style',
            'cultural fit', 'one of us', 'our type of person'
        ]

        self.coded_aggression = [
            'aggressive', 'intimidating', 'angry', 'hostile', 'chip on shoulder',
            'attitude problem', 'difficult', 'defensive'
        ]

        # Appearance bias
        self.appearance_words = [
            'professional appearance', 'well-groomed', 'presentable',
            'polished', 'image', 'looks the part', 'appearance',
            'dress code', 'personal presentation'
        ]

        # Vagueness (often masks unconscious bias)
        self.vague_patterns = [
            'not a good fit', 'cultural fit', 'gut feeling',
            'just feels off', 'something about', 'hard to put finger on',
            'just not right', 'concerns about', 'questions about'
        ]

    def detect_gender_bias(self, text: str) -> dict:
        """Detect gender bias (communal vs agentic language)"""
        text_lower = text.lower()

        communal_found = [w for w in self.communal_words if w in text_lower]
        agentic_found = [w for w in self.agentic_words if w in text_lower]

        communal_count = len(communal_found)
        agentic_count = len(agentic_found)

        # Calculate bias
        bias_detected = False
        bias_direction = None

        if communal_count > 0 and agentic_count > 0:
            ratio = max(communal_count, agentic_count) / min(communal_count, agentic_count)
            if ratio > 2.0:
                bias_detected = True
                bias_direction = 'communal' if communal_count > agentic_count else 'agentic'
        elif communal_count > 3:
            bias_detected = True
            bias_direction = 'communal'
        elif agentic_count > 3:
            bias_detected = True
            bias_direction = 'agentic'

        return {
            'detected': bias_detected,
            'direction': bias_direction,
            'communal_count': communal_count,
            'agentic_count': agentic_count,
            'communal_words': communal_found[:5],  # Show first 5
            'agentic_words': agentic_found[:5],
            'severity': 'high' if bias_detected and abs(communal_count - agentic_count) > 5 else 'moderate'
        }

    def detect_age_bias(self, text: str) -> dict:
        """Detect age-related bias"""
        text_lower = text.lower()

        young_found = [w for w in self.age_bias_young if w in text_lower]
        old_found = [w for w in self.age_bias_old if w in text_lower]

        young_count = len(young_found)
        old_count = len(old_found)

        bias_detected = (young_count + old_count) > 0

        if young_count > old_count:
            direction = 'younger'
        elif old_count > young_count:
            direction = 'older'
        else:
            direction = None

        return {
            'detected': bias_detected,
            'direction': direction,
            'young_indicators': young_count,
            'old_indicators': old_count,
            'young_words': young_found,
            'old_words': old_found
        }

    def detect_coded_language(self, text: str) -> dict:
        """Detect racially/ethnically coded language"""
        text_lower = text.lower()

        prof_found = [w for w in self.coded_professionalism if w in text_lower]
        culture_found = [w for w in self.coded_culture_fit if w in text_lower]
        aggr_found = [w for w in self.coded_aggression if w in text_lower]

        patterns_detected = []

        if prof_found:
            patterns_detected.append({
                'category': 'professionalism_coded',
                'words': prof_found,
                'risk': 'May imply surprise at professionalism'
            })

        if culture_found:
            patterns_detected.append({
                'category': 'culture_fit',
                'words': culture_found,
                'risk': 'Vague criteria that may mask bias'
            })

        if aggr_found:
            patterns_detected.append({
                'category': 'aggression_coded',
                'words': aggr_found,
                'risk': 'May reflect stereotyping'
            })

        return {
            'detected': len(patterns_detected) > 0,
            'pattern_count': len(patterns_detected),
            'patterns': patterns_detected
        }

    def detect_appearance_bias(self, text: str) -> dict:
        """Detect appearance-based bias"""
        text_lower = text.lower()

        found = [w for w in self.appearance_words if w in text_lower]

        return {
            'detected': len(found) > 0,
            'word_count': len(found),
            'words_found': found,
            'note': 'Appearance comments are often irrelevant to job performance'
        }

    def detect_vagueness(self, text: str) -> dict:
        """Detect vague language that often masks bias"""
        text_lower = text.lower()

        found = [p for p in self.vague_patterns if p in text_lower]

        return {
            'detected': len(found) > 0,
            'pattern_count': len(found),
            'patterns_found': found,
            'note': 'Vague statements lack specificity and may indicate unconscious bias'
        }

    def detect_all_bias_dimensions(self, text: str) -> dict:
        """
        Run comprehensive bias detection across all dimensions
        Returns detailed breakdown
        """
        results = {
            'text_length': len(text),
            'word_count': len(text.split()),
            'gender_bias': self.detect_gender_bias(text),
            'age_bias': self.detect_age_bias(text),
            'coded_language': self.detect_coded_language(text),
            'appearance_bias': self.detect_appearance_bias(text),
            'vagueness': self.detect_vagueness(text),
            'bias_types_detected': [],
            'overall_bias_score': 0.0,
            'risk_level': 'low'
        }

        # Calculate overall bias score (weighted)
        weights = {
            'gender_bias': 0.30,
            'age_bias': 0.20,
            'coded_language': 0.25,
            'appearance_bias': 0.10,
            'vagueness': 0.15
        }

        for bias_type, weight in weights.items():
            if results[bias_type]['detected']:
                results['bias_types_detected'].append(bias_type)
                results['overall_bias_score'] += weight

        # Determine risk level
        if results['overall_bias_score'] >= 0.5:
            results['risk_level'] = 'high'
        elif results['overall_bias_score'] >= 0.25:
            results['risk_level'] = 'moderate'
        else:
            results['risk_level'] = 'low'

        return results

print("✅ EnhancedBiasDetector class loaded successfully!")
print(f"   Dimensions: Gender, Age, Coded Language, Appearance, Vagueness")

✅ EnhancedBiasDetector class loaded successfully!
   Dimensions: Gender, Age, Coded Language, Appearance, Vagueness


In [44]:
# Test the enhanced detector on your sample reviews

enhanced_detector = EnhancedBiasDetector()

print("="*70)
print("ENHANCED MULTI-DIMENSIONAL BIAS DETECTION TEST")
print("="*70)

# Test on Martha Barnes review (communal bias)
print("\n" + "="*70)
print("TEST 1: Martha Barnes Review")
print("="*70)

martha_file = 'sample_reviews/review_martha_barnes.pdf'
martha_doc = doc_processor.process_document(martha_file)
martha_results = enhanced_detector.detect_all_bias_dimensions(martha_doc['text'])

print(f"\n📊 OVERALL RESULTS:")
print(f"   Risk Level: {martha_results['risk_level'].upper()}")
print(f"   Bias Score: {martha_results['overall_bias_score']:.2f}")
print(f"   Dimensions Detected: {len(martha_results['bias_types_detected'])}")

if martha_results['gender_bias']['detected']:
    print(f"\n⚠️  GENDER BIAS DETECTED:")
    print(f"   Direction: {martha_results['gender_bias']['direction']}")
    print(f"   Communal: {martha_results['gender_bias']['communal_count']} words")
    print(f"   Agentic: {martha_results['gender_bias']['agentic_count']} words")
    print(f"   Examples: {', '.join(martha_results['gender_bias']['communal_words'][:3])}")

if martha_results['vagueness']['detected']:
    print(f"\n⚠️  VAGUENESS DETECTED:")
    print(f"   Patterns: {', '.join(martha_results['vagueness']['patterns_found'])}")

# Test on James Wilson review (agentic bias)
print("\n" + "="*70)
print("TEST 2: James Wilson Review")
print("="*70)

james_file = 'sample_reviews/review_james_wilson.pdf'
james_doc = doc_processor.process_document(james_file)
james_results = enhanced_detector.detect_all_bias_dimensions(james_doc['text'])

print(f"\n📊 OVERALL RESULTS:")
print(f"   Risk Level: {james_results['risk_level'].upper()}")
print(f"   Bias Score: {james_results['overall_bias_score']:.2f}")
print(f"   Dimensions Detected: {len(james_results['bias_types_detected'])}")

if james_results['gender_bias']['detected']:
    print(f"\n⚠️  GENDER BIAS DETECTED:")
    print(f"   Direction: {james_results['gender_bias']['direction']}")
    print(f"   Communal: {james_results['gender_bias']['communal_count']} words")
    print(f"   Agentic: {james_results['gender_bias']['agentic_count']} words")
    print(f"   Examples: {', '.join(james_results['gender_bias']['agentic_words'][:3])}")

# Test on Alex Chen review (neutral)
print("\n" + "="*70)
print("TEST 3: Alex Chen Review (Should be clean)")
print("="*70)

alex_file = 'sample_reviews/review_alex_chen.pdf'
alex_doc = doc_processor.process_document(alex_file)
alex_results = enhanced_detector.detect_all_bias_dimensions(alex_doc['text'])

print(f"\n📊 OVERALL RESULTS:")
print(f"   Risk Level: {alex_results['risk_level'].upper()}")
print(f"   Bias Score: {alex_results['overall_bias_score']:.2f}")
print(f"   Dimensions Detected: {len(alex_results['bias_types_detected'])}")

if alex_results['bias_types_detected']:
    print(f"\n   Bias types: {', '.join(alex_results['bias_types_detected'])}")
else:
    print(f"\n✅ NO SIGNIFICANT BIAS DETECTED - Review appears fair and objective")

ENHANCED MULTI-DIMENSIONAL BIAS DETECTION TEST

TEST 1: Martha Barnes Review

📊 OVERALL RESULTS:
   Risk Level: HIGH
   Bias Score: 0.55
   Dimensions Detected: 2

⚠️  GENDER BIAS DETECTED:
   Direction: communal
   Communal: 7 words
   Agentic: 3 words
   Examples: collaborative, pleasant, supportive

TEST 2: James Wilson Review

📊 OVERALL RESULTS:
   Risk Level: MODERATE
   Bias Score: 0.30
   Dimensions Detected: 1

⚠️  GENDER BIAS DETECTED:
   Direction: agentic
   Communal: 0 words
   Agentic: 14 words
   Examples: leader, leadership, drives

TEST 3: Alex Chen Review (Should be clean)

📊 OVERALL RESULTS:
   Risk Level: LOW
   Bias Score: 0.00
   Dimensions Detected: 0

✅ NO SIGNIFICANT BIAS DETECTED - Review appears fair and objective


In [45]:
# Create a simple comparison table

import pandas as pd

# Collect results for all three reviews
comparison_data = []

for name, file_path in [
    ('Martha Barnes', 'sample_reviews/review_martha_barnes.pdf'),
    ('James Wilson', 'sample_reviews/review_james_wilson.pdf'),
    ('Alex Chen', 'sample_reviews/review_alex_chen.pdf')
]:
    doc = doc_processor.process_document(file_path)
    results = enhanced_detector.detect_all_bias_dimensions(doc['text'])

    comparison_data.append({
        'Employee': name,
        'Risk Level': results['risk_level'],
        'Bias Score': f"{results['overall_bias_score']:.2f}",
        'Gender Bias': '✓' if results['gender_bias']['detected'] else '✗',
        'Age Bias': '✓' if results['age_bias']['detected'] else '✗',
        'Coded Language': '✓' if results['coded_language']['detected'] else '✗',
        'Vagueness': '✓' if results['vagueness']['detected'] else '✗',
        'Total Dimensions': len(results['bias_types_detected'])
    })

df_comparison = pd.DataFrame(comparison_data)

print("\n" + "="*70)
print("BIAS DETECTION COMPARISON - ALL REVIEWS")
print("="*70)
print(df_comparison.to_string(index=False))

print("\n📊 SUMMARY:")
print(f"   High Risk Reviews: {len([x for x in comparison_data if x['Risk Level'] == 'high'])}")
print(f"   Moderate Risk Reviews: {len([x for x in comparison_data if x['Risk Level'] == 'moderate'])}")
print(f"   Low Risk Reviews: {len([x for x in comparison_data if x['Risk Level'] == 'low'])}")


BIAS DETECTION COMPARISON - ALL REVIEWS
     Employee Risk Level Bias Score Gender Bias Age Bias Coded Language Vagueness  Total Dimensions
Martha Barnes       high       0.55           ✓        ✗              ✓         ✗                 2
 James Wilson   moderate       0.30           ✓        ✗              ✗         ✗                 1
    Alex Chen        low       0.00           ✗        ✗              ✗         ✗                 0

📊 SUMMARY:
   High Risk Reviews: 1
   Moderate Risk Reviews: 1
   Low Risk Reviews: 1


In [10]:
import logging
import json
from datetime import datetime
import uuid

class AuditLogger:
    """
    SOC 2 compliant audit logging for bias detection system
    Logs all operations with immutable audit trail
    """

    def __init__(self, log_file='bias_detection_audit.log'):
        self.log_file = log_file

        # Configure structured logging (JSON format)
        logging.basicConfig(
            level=logging.INFO,
            format='%(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()  # Also print to console
            ]
        )
        self.logger = logging.getLogger(__name__)

        # Initialize session
        self.session_id = str(uuid.uuid4())[:8]
        self.log_count = 0

        self._log_session_start()

    def _log_session_start(self):
        """Log the start of a new session"""
        log_entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'session_id': self.session_id,
            'event_type': 'SESSION_START',
            'message': 'Bias detection system initialized'
        }
        self.logger.info(json.dumps(log_entry))
        self.log_count += 1

    def log_document_processing(self, file_name: str, file_format: str,
                                success: bool, word_count: int = 0,
                                error: str = None):
        """Log document extraction event"""
        log_entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'session_id': self.session_id,
            'log_id': self.log_count,
            'event_type': 'DOCUMENT_PROCESSING',
            'file_name': file_name,
            'file_format': file_format,
            'success': success,
            'word_count': word_count,
            'error': error
        }
        self.logger.info(json.dumps(log_entry))
        self.log_count += 1

    def log_bias_detection(self, file_name: str, bias_results: dict):
        """Log bias detection results"""
        log_entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'session_id': self.session_id,
            'log_id': self.log_count,
            'event_type': 'BIAS_DETECTION',
            'file_name': file_name,
            'risk_level': bias_results.get('risk_level'),
            'bias_score': bias_results.get('overall_bias_score'),
            'dimensions_detected': len(bias_results.get('bias_types_detected', [])),
            'bias_types': bias_results.get('bias_types_detected', [])
        }
        self.logger.info(json.dumps(log_entry))
        self.log_count += 1

    def log_security_event(self, event_type: str, details: dict, severity: str = 'INFO'):
        """Log security-relevant events"""
        log_entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'session_id': self.session_id,
            'log_id': self.log_count,
            'event_type': 'SECURITY_EVENT',
            'security_event_type': event_type,
            'severity': severity,
            'details': details
        }

        if severity == 'HIGH':
            self.logger.warning(json.dumps(log_entry))
        else:
            self.logger.info(json.dumps(log_entry))

        self.log_count += 1

    def log_rag_query(self, query: str, documents_retrieved: int, success: bool):
        """Log RAG system queries"""
        log_entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'session_id': self.session_id,
            'log_id': self.log_count,
            'event_type': 'RAG_QUERY',
            'query_preview': query[:100],  # First 100 chars only
            'documents_retrieved': documents_retrieved,
            'success': success
        }
        self.logger.info(json.dumps(log_entry))
        self.log_count += 1

    def log_llm_rewrite(self, file_name: str, original_bias_score: float,
                       new_bias_score: float, success: bool):
        """Log LLM rewrite operations"""
        improvement = original_bias_score - new_bias_score

        log_entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'session_id': self.session_id,
            'log_id': self.log_count,
            'event_type': 'LLM_REWRITE',
            'file_name': file_name,
            'original_bias_score': original_bias_score,
            'new_bias_score': new_bias_score,
            'improvement': improvement,
            'success': success
        }
        self.logger.info(json.dumps(log_entry))
        self.log_count += 1

    def get_session_summary(self) -> dict:
        """Get summary of current session"""
        return {
            'session_id': self.session_id,
            'total_logs': self.log_count,
            'log_file': self.log_file
        }

    def log_session_end(self):
        """Log the end of session"""
        log_entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'session_id': self.session_id,
            'event_type': 'SESSION_END',
            'total_operations': self.log_count,
            'message': 'Session completed'
        }
        self.logger.info(json.dumps(log_entry))
        self.log_count += 1

print("✅ AuditLogger class loaded successfully!")
print("   Features: Structured JSON logs, session tracking, SOC 2 compliance")

✅ AuditLogger class loaded successfully!
   Features: Structured JSON logs, session tracking, SOC 2 compliance


In [46]:
# Initialize audit logger
audit_logger = AuditLogger(log_file='bias_audit_demo.log')

print("="*70)
print("TESTING AUDIT LOGGING SYSTEM")
print("="*70)
print(f"\nSession ID: {audit_logger.session_id}")
print(f"Log file: {audit_logger.log_file}\n")

# Test logging for document processing
print("[TEST 1] Document Processing Logs")
print("-"*70)

audit_logger.log_document_processing(
    file_name='review_martha_barnes.pdf',
    file_format='.pdf',
    success=True,
    word_count=103
)
print("✅ Logged: Document processing (success)")

audit_logger.log_document_processing(
    file_name='corrupted_file.pdf',
    file_format='.pdf',
    success=False,
    error='File not readable'
)
print("✅ Logged: Document processing (failure)")

# Test logging for bias detection
print("\n[TEST 2] Bias Detection Logs")
print("-"*70)

# Create sample bias results
sample_bias_results = {
    'risk_level': 'high',
    'overall_bias_score': 0.55,
    'bias_types_detected': ['gender_bias', 'vagueness']
}

audit_logger.log_bias_detection(
    file_name='review_martha_barnes.pdf',
    bias_results=sample_bias_results
)
print("✅ Logged: Bias detection results")

# Test security event logging
print("\n[TEST 3] Security Event Logs")
print("-"*70)

audit_logger.log_security_event(
    event_type='INPUT_VALIDATION',
    details={'file': 'suspicious_file.exe', 'reason': 'Invalid file format'},
    severity='HIGH'
)
print("✅ Logged: Security event (HIGH severity)")

# Test RAG query logging
print("\n[TEST 4] RAG Query Logs")
print("-"*70)

audit_logger.log_rag_query(
    query='EEOC guidelines on gender discrimination in performance reviews',
    documents_retrieved=5,
    success=True
)
print("✅ Logged: RAG query")

# Test LLM rewrite logging
print("\n[TEST 5] LLM Rewrite Logs")
print("-"*70)

audit_logger.log_llm_rewrite(
    file_name='review_martha_barnes.pdf',
    original_bias_score=0.55,
    new_bias_score=0.15,
    success=True
)
print("✅ Logged: LLM rewrite operation")

# End session
audit_logger.log_session_end()

# Print summary
print("\n" + "="*70)
print("SESSION SUMMARY")
print("="*70)
summary = audit_logger.get_session_summary()
print(f"Session ID: {summary['session_id']}")
print(f"Total Logs: {summary['total_logs']}")
print(f"Log File: {summary['log_file']}")

/tmp/ipython-input-250486379.py:35: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:64: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. U

TESTING AUDIT LOGGING SYSTEM

Session ID: ff6da885
Log file: bias_audit_demo.log

[TEST 1] Document Processing Logs
----------------------------------------------------------------------
✅ Logged: Document processing (success)
✅ Logged: Document processing (failure)

[TEST 2] Bias Detection Logs
----------------------------------------------------------------------
✅ Logged: Bias detection results

[TEST 3] Security Event Logs
----------------------------------------------------------------------
✅ Logged: Security event (HIGH severity)

[TEST 4] RAG Query Logs
----------------------------------------------------------------------
✅ Logged: RAG query

[TEST 5] LLM Rewrite Logs
----------------------------------------------------------------------
✅ Logged: LLM rewrite operation

SESSION SUMMARY
Session ID: ff6da885
Total Logs: 8
Log File: bias_audit_demo.log


/tmp/ipython-input-250486379.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:116: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:140: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),


In [47]:
def process_document_with_full_logging(file_path: str):
    """
    Complete pipeline with comprehensive audit logging
    Document → Extract → Detect → Log
    """

    # Initialize logger for this run
    logger = AuditLogger(log_file='production_audit.log')

    print(f"\n{'='*70}")
    print(f"PRODUCTION PIPELINE WITH AUDIT LOGGING")
    print(f"{'='*70}")
    print(f"Session ID: {logger.session_id}")
    print(f"File: {file_path}\n")

    # Step 1: Document Processing
    print("[STEP 1] Document Processing...")
    doc_processor = DocumentProcessor()

    try:
        extracted = doc_processor.process_document(file_path)

        # Log successful extraction
        logger.log_document_processing(
            file_name=extracted['file_name'],
            file_format=extracted['format'],
            success=True,
            word_count=extracted['word_count']
        )

        print(f"✅ Extracted {extracted['word_count']} words")

    except Exception as e:
        # Log failed extraction
        logger.log_document_processing(
            file_name=Path(file_path).name,
            file_format=Path(file_path).suffix,
            success=False,
            error=str(e)
        )
        print(f"❌ Extraction failed: {str(e)}")
        logger.log_session_end()
        return None

    # Step 2: Enhanced Bias Detection
    print("\n[STEP 2] Multi-Dimensional Bias Detection...")
    detector = EnhancedBiasDetector()

    bias_results = detector.detect_all_bias_dimensions(extracted['text'])

    # Log bias detection
    logger.log_bias_detection(
        file_name=extracted['file_name'],
        bias_results=bias_results
    )

    print(f"✅ Analysis complete")
    print(f"   Risk Level: {bias_results['risk_level'].upper()}")
    print(f"   Bias Score: {bias_results['overall_bias_score']:.2f}")
    print(f"   Dimensions: {len(bias_results['bias_types_detected'])}")

    # Step 3: Results Summary
    print(f"\n[STEP 3] Results Summary...")

    if bias_results['bias_types_detected']:
        print(f"⚠️  BIAS DETECTED in {len(bias_results['bias_types_detected'])} dimensions:")
        for bias_type in bias_results['bias_types_detected']:
            print(f"   • {bias_type.replace('_', ' ').title()}")
    else:
        print(f"✅ NO SIGNIFICANT BIAS DETECTED")

    # End session
    logger.log_session_end()

    print(f"\n{'='*70}")
    print(f"AUDIT TRAIL COMPLETE")
    print(f"{'='*70}")
    print(f"Total operations logged: {logger.log_count}")
    print(f"Log file: {logger.log_file}")

    return {
        'file': extracted['file_name'],
        'bias_results': bias_results,
        'session_id': logger.session_id
    }

# Test on all sample reviews
print("\n" + "="*70)
print("FULL PRODUCTION PIPELINE TEST (WITH LOGGING)")
print("="*70)

for file_path in [
    'sample_reviews/review_martha_barnes.pdf',
    'sample_reviews/review_james_wilson.pdf',
    'sample_reviews/review_alex_chen.pdf'
]:
    result = process_document_with_full_logging(file_path)
    print("\n" + "-"*70 + "\n")


FULL PRODUCTION PIPELINE TEST (WITH LOGGING)

PRODUCTION PIPELINE WITH AUDIT LOGGING
Session ID: c64215cf
File: sample_reviews/review_martha_barnes.pdf

[STEP 1] Document Processing...
✅ Extracted 103 words

[STEP 2] Multi-Dimensional Bias Detection...
✅ Analysis complete
   Risk Level: HIGH
   Bias Score: 0.55
   Dimensions: 2

[STEP 3] Results Summary...
⚠️  BIAS DETECTED in 2 dimensions:
   • Gender Bias
   • Coded Language

AUDIT TRAIL COMPLETE
Total operations logged: 4
Log file: production_audit.log

----------------------------------------------------------------------


PRODUCTION PIPELINE WITH AUDIT LOGGING
Session ID: 5b25365a
File: sample_reviews/review_james_wilson.pdf

[STEP 1] Document Processing...
✅ Extracted 99 words

[STEP 2] Multi-Dimensional Bias Detection...
✅ Analysis complete
   Risk Level: MODERATE
   Bias Score: 0.30
   Dimensions: 1

[STEP 3] Results Summary...
⚠️  BIAS DETECTED in 1 dimensions:
   • Gender Bias

AUDIT TRAIL COMPLETE
Total operations logged: 

/tmp/ipython-input-250486379.py:35: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:64: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:140: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. 

In [48]:
# Read and display the audit log in a nice format

import json

def display_audit_log(log_file='production_audit.log', max_entries=20):
    """Display audit log in readable format"""

    print("="*70)
    print(f"AUDIT LOG VIEWER: {log_file}")
    print("="*70)

    try:
        with open(log_file, 'r') as f:
            lines = f.readlines()

        print(f"\nTotal log entries: {len(lines)}")
        print(f"Showing last {min(max_entries, len(lines))} entries:\n")

        # Show last N entries
        for i, line in enumerate(lines[-max_entries:], 1):
            try:
                log_entry = json.loads(line)

                timestamp = log_entry.get('timestamp', 'N/A')[:19]  # Remove milliseconds
                event_type = log_entry.get('event_type', 'UNKNOWN')

                print(f"[{i}] {timestamp} | {event_type}")

                # Show relevant details based on event type
                if event_type == 'DOCUMENT_PROCESSING':
                    print(f"    File: {log_entry.get('file_name')}")
                    print(f"    Success: {log_entry.get('success')}")
                    print(f"    Words: {log_entry.get('word_count', 'N/A')}")

                elif event_type == 'BIAS_DETECTION':
                    print(f"    File: {log_entry.get('file_name')}")
                    print(f"    Risk: {log_entry.get('risk_level')}")
                    print(f"    Score: {log_entry.get('bias_score')}")
                    print(f"    Types: {', '.join(log_entry.get('bias_types', []))}")

                elif event_type == 'SECURITY_EVENT':
                    print(f"    Type: {log_entry.get('security_event_type')}")
                    print(f"    Severity: {log_entry.get('severity')}")

                elif event_type == 'RAG_QUERY':
                    print(f"    Query: {log_entry.get('query_preview')}")
                    print(f"    Docs: {log_entry.get('documents_retrieved')}")

                elif event_type == 'LLM_REWRITE':
                    print(f"    File: {log_entry.get('file_name')}")
                    print(f"    Before: {log_entry.get('original_bias_score')}")
                    print(f"    After: {log_entry.get('new_bias_score')}")
                    print(f"    Improvement: {log_entry.get('improvement'):.2f}")

                print()

            except json.JSONDecodeError:
                print(f"[{i}] Invalid JSON entry")
                print()

    except FileNotFoundError:
        print(f"❌ Log file not found: {log_file}")
        print("   Run the pipeline first to generate logs.")

# Display the production audit log
display_audit_log('production_audit.log', max_entries=30)

AUDIT LOG VIEWER: production_audit.log

Total log entries: 0
Showing last 0 entries:



In [49]:
def enhanced_pipeline_v2(file_path: str):
    """
    ENHANCED VERSION 2: Complete Production Pipeline

    Pipeline Flow:
    1. Extract text from PDF/DOCX/TXT
    2. Multi-dimensional bias detection (5 dimensions)
    3. Audit logging (SOC 2 compliance)
    4. Return comprehensive results

    Args:
        file_path: Path to document (PDF, DOCX, or TXT)

    Returns:
        dict: Complete analysis results with audit trail
    """

    from pathlib import Path

    print("\n" + "="*70)
    print("ENHANCED PIPELINE V2 - PRODUCTION-READY BIAS DETECTION")
    print("="*70)
    print(f"Document: {Path(file_path).name}")
    print(f"Timestamp: {datetime.utcnow().isoformat()}")

    # Initialize components
    logger = AuditLogger(log_file=f'audit_{Path(file_path).stem}.log')
    doc_processor = DocumentProcessor()
    enhanced_detector = EnhancedBiasDetector()

    print(f"Session ID: {logger.session_id}\n")

    # ===================================================================
    # STEP 1: DOCUMENT EXTRACTION
    # ===================================================================
    print("[STEP 1] DOCUMENT PROCESSING")
    print("-"*70)

    try:
        extracted = doc_processor.process_document(file_path)

        # Log successful extraction
        logger.log_document_processing(
            file_name=extracted['file_name'],
            file_format=extracted['format'],
            success=True,
            word_count=extracted['word_count']
        )

        print(f"✅ Document Type: {extracted['format']}")
        print(f"✅ Word Count: {extracted['word_count']}")
        print(f"✅ Character Count: {extracted['char_count']}")
        print(f"📝 Preview: {extracted['text'][:150]}...\n")

    except Exception as e:
        # Log failed extraction
        logger.log_document_processing(
            file_name=Path(file_path).name,
            file_format=Path(file_path).suffix,
            success=False,
            error=str(e)
        )

        print(f"❌ EXTRACTION FAILED: {str(e)}\n")
        logger.log_session_end()
        return None

    # ===================================================================
    # STEP 2: MULTI-DIMENSIONAL BIAS DETECTION
    # ===================================================================
    print("[STEP 2] MULTI-DIMENSIONAL BIAS ANALYSIS")
    print("-"*70)

    bias_results = enhanced_detector.detect_all_bias_dimensions(extracted['text'])

    # Log bias detection results
    logger.log_bias_detection(
        file_name=extracted['file_name'],
        bias_results=bias_results
    )

    # Display overall results
    print(f"📊 OVERALL ASSESSMENT:")
    print(f"   Risk Level: {bias_results['risk_level'].upper()}")
    print(f"   Bias Score: {bias_results['overall_bias_score']:.2f} / 1.00")
    print(f"   Dimensions Flagged: {len(bias_results['bias_types_detected'])}")

    # Display dimension-by-dimension breakdown
    if bias_results['bias_types_detected']:
        print(f"\n⚠️  BIAS DETECTED IN {len(bias_results['bias_types_detected'])} DIMENSION(S):\n")

        # Gender Bias Details
        if 'gender_bias' in bias_results['bias_types_detected']:
            gb = bias_results['gender_bias']
            print(f"   1️⃣  GENDER BIAS ({gb['severity']})")
            print(f"      Direction: {gb['direction']}")
            print(f"      Communal words: {gb['communal_count']} ({', '.join(gb['communal_words'][:3])})")
            print(f"      Agentic words: {gb['agentic_count']} ({', '.join(gb['agentic_words'][:3])})")

        # Age Bias Details
        if 'age_bias' in bias_results['bias_types_detected']:
            ab = bias_results['age_bias']
            print(f"\n   2️⃣  AGE BIAS")
            print(f"      Direction: {ab['direction']}")
            print(f"      Indicators: {', '.join(ab['young_words'] + ab['old_words'])}")

        # Coded Language Details
        if 'coded_language' in bias_results['bias_types_detected']:
            cl = bias_results['coded_language']
            print(f"\n   3️⃣  CODED LANGUAGE")
            print(f"      Patterns found: {cl['pattern_count']}")
            for pattern in cl['patterns']:
                print(f"      • {pattern['category']}: {', '.join(pattern['words'])}")
                print(f"        Risk: {pattern['risk']}")

        # Appearance Bias Details
        if 'appearance_bias' in bias_results['bias_types_detected']:
            ap = bias_results['appearance_bias']
            print(f"\n   4️⃣  APPEARANCE BIAS")
            print(f"      Words: {', '.join(ap['words_found'])}")
            print(f"      Note: {ap['note']}")

        # Vagueness Details
        if 'vagueness' in bias_results['bias_types_detected']:
            vg = bias_results['vagueness']
            print(f"\n   5️⃣  VAGUENESS")
            print(f"      Patterns: {', '.join(vg['patterns_found'])}")
            print(f"      Note: {vg['note']}")

    else:
        print(f"\n✅ NO SIGNIFICANT BIAS DETECTED")
        print(f"   Review appears fair and objective\n")

    # ===================================================================
    # STEP 3: RECOMMENDATIONS
    # ===================================================================
    print("\n[STEP 3] RECOMMENDATIONS")
    print("-"*70)

    if bias_results['risk_level'] == 'high':
        print("🚨 HIGH RISK - IMMEDIATE ACTION REQUIRED:")
        print("   • Flag for HR review")
        print("   • Consider rewriting with neutral language")
        print("   • Manager coaching recommended")
        print("   • Legal compliance review suggested")
    elif bias_results['risk_level'] == 'moderate':
        print("⚠️  MODERATE RISK - ATTENTION NEEDED:")
        print("   • Review with manager")
        print("   • Consider language adjustments")
        print("   • Monitor for patterns")
    else:
        print("✅ LOW RISK - ACCEPTABLE:")
        print("   • Review meets fairness standards")
        print("   • No immediate action required")

    # ===================================================================
    # STEP 4: AUDIT TRAIL
    # ===================================================================
    logger.log_session_end()

    print(f"\n[STEP 4] AUDIT TRAIL")
    print("-"*70)
    print(f"✅ All operations logged")
    print(f"📁 Log file: {logger.log_file}")
    print(f"📊 Total operations: {logger.log_count}")
    print(f"🔒 Immutable audit trail for compliance")

    # ===================================================================
    # FINAL OUTPUT
    # ===================================================================
    print("\n" + "="*70)
    print("PIPELINE COMPLETE")
    print("="*70)

    return {
        'file_name': extracted['file_name'],
        'file_format': extracted['format'],
        'word_count': extracted['word_count'],
        'extracted_text': extracted['text'],
        'bias_analysis': bias_results,
        'risk_level': bias_results['risk_level'],
        'bias_score': bias_results['overall_bias_score'],
        'dimensions_detected': bias_results['bias_types_detected'],
        'session_id': logger.session_id,
        'audit_log': logger.log_file,
        'timestamp': datetime.utcnow().isoformat()
    }

print("✅ Enhanced Pipeline V2 loaded successfully!")
print("   Features: Document processing + 5D bias detection + Audit logging")

✅ Enhanced Pipeline V2 loaded successfully!
   Features: Document processing + 5D bias detection + Audit logging


In [50]:


# Test the complete enhanced pipeline on all 3 sample reviews

print("\n" + "="*70)
print("TESTING ENHANCED PIPELINE V2 - ALL SAMPLE REVIEWS")
print("="*70)

test_files = [
    'sample_reviews/review_martha_barnes.pdf',
    'sample_reviews/review_james_wilson.pdf',
    'sample_reviews/review_alex_chen.pdf'
]

results_summary = []

for file_path in test_files:
    result = enhanced_pipeline_v2(file_path)

    if result:
        results_summary.append({
            'File': result['file_name'],
            'Risk': result['risk_level'],
            'Score': f"{result['bias_score']:.2f}",
            'Dimensions': len(result['dimensions_detected']),
            'Types': ', '.join([d.replace('_', ' ').title() for d in result['dimensions_detected']])
        })

    print("\n" + "="*70 + "\n")
    input("Press Enter to continue to next review...")

# Summary table
print("\n" + "="*70)
print("BATCH PROCESSING SUMMARY")
print("="*70)

import pandas as pd
df_summary = pd.DataFrame(results_summary)
print(df_summary.to_string(index=False))

print(f"\n📊 STATISTICS:")
print(f"   Total reviews processed: {len(results_summary)}")
print(f"   High risk: {len([r for r in results_summary if r['Risk'] == 'high'])}")
print(f"   Moderate risk: {len([r for r in results_summary if r['Risk'] == 'moderate'])}")
print(f"   Low risk: {len([r for r in results_summary if r['Risk'] == 'low'])}")

/tmp/ipython-input-3390770164.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"Timestamp: {datetime.utcnow().isoformat()}")
/tmp/ipython-input-250486379.py:35: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(),
/tmp/ipython-input-250486379.py:64: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future 


TESTING ENHANCED PIPELINE V2 - ALL SAMPLE REVIEWS

ENHANCED PIPELINE V2 - PRODUCTION-READY BIAS DETECTION
Document: review_martha_barnes.pdf
Timestamp: 2025-12-21T17:51:33.338744
Session ID: 9d34ecdf

[STEP 1] DOCUMENT PROCESSING
----------------------------------------------------------------------
✅ Document Type: .pdf
✅ Word Count: 103
✅ Character Count: 704
📝 Preview: ANNUAL PERFORMANCE REVIEW Employee: Martha Barnes - Senior Analyst Review Period: January - December 2024 Reviewer: Manager - Operations Performance S...

[STEP 2] MULTI-DIMENSIONAL BIAS ANALYSIS
----------------------------------------------------------------------
📊 OVERALL ASSESSMENT:
   Risk Level: HIGH
   Bias Score: 0.55 / 1.00
   Dimensions Flagged: 2

⚠️  BIAS DETECTED IN 2 DIMENSION(S):

   1️⃣  GENDER BIAS (moderate)
      Direction: communal
      Communal words: 7 (collaborative, pleasant, supportive)
      Agentic words: 3 (assertive, confident, excellent)

   3️⃣  CODED LANGUAGE
      Patterns found: 1


In [17]:
def demo_side_by_side_comparison():
    """
    Compare original system vs enhanced system
    Shows the evolution for interview demo
    """

    test_file = 'sample_reviews/review_martha_barnes.pdf'

    print("\n" + "="*70)
    print("SIDE-BY-SIDE COMPARISON: V1 vs V2")
    print("="*70)
    print(f"Test Document: {test_file}\n")

    # Extract document first
    doc_processor = DocumentProcessor()
    extracted = doc_processor.process_document(test_file)
    text = extracted['text']

    # ===================================================================
    # ORIGINAL SYSTEM (V1)
    # ===================================================================
    print("┌" + "─"*68 + "┐")
    print("│" + " "*20 + "ORIGINAL SYSTEM (V1)" + " "*27 + "│")
    print("└" + "─"*68 + "┘")
    print("\n📌 Features:")
    print("   • Basic gender bias detection (communal vs agentic)")
    print("   • Works on CSV data only")
    print("   • Simple word counting\n")

    # Simple gender bias check (your original approach)
    communal_words = ['collaborative', 'pleasant', 'supportive', 'helpful',
                     'nurturing', 'empathy', 'team player']
    agentic_words = ['leadership', 'drives', 'assertive', 'confident',
                    'strategic', 'initiative', 'decisive']

    text_lower = text.lower()
    communal_count = sum(1 for w in communal_words if w in text_lower)
    agentic_count = sum(1 for w in agentic_words if w in text_lower)

    print(f"📊 V1 Results:")
    print(f"   Communal words: {communal_count}")
    print(f"   Agentic words: {agentic_count}")

    if communal_count > agentic_count * 2:
        print(f"   ⚠️  Bias detected: HIGH communal language")
    else:
        print(f"   Status: Unclear/Ambiguous")

    print()

    # ===================================================================
    # ENHANCED SYSTEM (V2)
    # ===================================================================
    print("┌" + "─"*68 + "┐")
    print("│" + " "*19 + "ENHANCED SYSTEM (V2)" + " "*28 + "│")
    print("└" + "─"*68 + "┘")
    print("\n📌 NEW Features:")
    print("   • Multi-format support (PDF, DOCX, TXT)")
    print("   • 5-dimensional bias detection")
    print("   • Production audit logging")
    print("   • Risk scoring & recommendations\n")

    enhanced_detector = EnhancedBiasDetector()
    v2_results = enhanced_detector.detect_all_bias_dimensions(text)

    print(f"📊 V2 Results:")
    print(f"   Risk Level: {v2_results['risk_level'].upper()}")
    print(f"   Overall Bias Score: {v2_results['overall_bias_score']:.2f}")
    print(f"   Dimensions Detected: {len(v2_results['bias_types_detected'])}")
    print(f"   Types: {', '.join([d.replace('_', ' ').title() for d in v2_results['bias_types_detected']])}")

    # ===================================================================
    # COMPARISON SUMMARY
    # ===================================================================
    print("\n" + "="*70)
    print("IMPROVEMENT SUMMARY")
    print("="*70)

    improvements = [
        "✅ Handles real PDFs/DOCX (not just CSV)",
        "✅ Detects 5 bias types (not just gender)",
        "✅ Quantifiable risk score (0.0 - 1.0)",
        "✅ Production-ready logging",
        "✅ Actionable recommendations",
        "✅ SOC 2 audit compliance"
    ]

    for improvement in improvements:
        print(f"   {improvement}")

    print("\n💡 This evolution shows:")
    print("   • Listened to Pedestal's needs (unstructured data)")
    print("   • Production mindset (logging, compliance)")
    print("   • Scalable architecture (5 dimensions vs 1)")

# Run the comparison
demo_side_by_side_comparison()


SIDE-BY-SIDE COMPARISON: V1 vs V2
Test Document: sample_reviews/review_martha_barnes.pdf

┌────────────────────────────────────────────────────────────────────┐
│                    ORIGINAL SYSTEM (V1)                           │
└────────────────────────────────────────────────────────────────────┘

📌 Features:
   • Basic gender bias detection (communal vs agentic)
   • Works on CSV data only
   • Simple word counting

📊 V1 Results:
   Communal words: 7
   Agentic words: 2
   ⚠️  Bias detected: HIGH communal language

┌────────────────────────────────────────────────────────────────────┐
│                   ENHANCED SYSTEM (V2)                            │
└────────────────────────────────────────────────────────────────────┘

📌 NEW Features:
   • Multi-format support (PDF, DOCX, TXT)
   • 5-dimensional bias detection
   • Production audit logging
   • Risk scoring & recommendations

📊 V2 Results:
   Risk Level: HIGH
   Overall Bias Score: 0.55
   Dimensions Detected: 2
   Types: Ge

In [38]:
# Cell 38: Expand RAG with New Bias Type Documents

print("="*70)
print("EXPANDING RAG KNOWLEDGE BASE")
print("="*70)

# New documents for age bias
age_bias_doc = """
AGE DISCRIMINATION IN EMPLOYMENT ACT (ADEA) - KEY POINTS

The ADEA protects individuals 40 years of age or older from employment discrimination.

PROHIBITED LANGUAGE IN REVIEWS:
- References to "old school" methods or "traditional" approaches
- Comments about being "set in ways" or "resistant to change"
- Descriptions like "seasoned veteran" that emphasize age over performance
- Assumptions about technology adoption based on age
- Terms like "overqualified" that may signal age discrimination

LEGAL REQUIREMENTS:
- Performance evaluations must focus on job-related criteria
- Age-neutral language required for all performance discussions
- Documented evidence of performance issues required
- No assumptions about retirement plans or tenure

CASE LAW:
- Smith v. City of Jackson (2005): Disparate impact applies to ADEA
- Gross v. FBL Financial Services (2009): Age must be "but-for" cause
"""

# New documents for coded language
coded_language_doc = """
IMPLICIT BIAS AND CODED LANGUAGE IN PERFORMANCE REVIEWS

CODED PROFESSIONALISM:
Terms like "articulate" or "well-spoken" when applied to minorities may indicate
surprise at professionalism, suggesting bias.

LEGAL STANDARD: Focus on specific job performance, not subjective impressions.

CULTURE FIT CONCERNS:
Phrases like "not a good fit" or "different style" often mask discriminatory intent.
Courts scrutinize vague criteria that disproportionately affect protected groups.

APPEARANCE-BASED COMMENTS:
Comments on personal appearance, grooming, or "professional image" must be directly
job-related and applied consistently across all employees.

VAGUE LANGUAGE RED FLAGS:
- "Gut feeling" or "just feels off"
- "Something about them" without specifics
- "Concerns" without documented evidence
- "Questions about" without concrete examples

BEST PRACTICE: Replace vague language with specific, measurable, job-related criteria.
"""

# New documents for comprehensive debiasing
debiasing_best_practices = """
EVIDENCE-BASED PERFORMANCE REVIEW BEST PRACTICES

OBJECTIVE CRITERIA:
- Use specific metrics and measurable outcomes
- Compare against clearly defined job expectations
- Document specific examples with dates and context
- Focus on behaviors and results, not personality traits

BALANCED LANGUAGE:
- Use same evaluative standards for all employees
- Avoid gendered language (communal vs agentic)
- Eliminate age-related assumptions
- Remove appearance-based comments unless job-related

LEGAL COMPLIANCE:
- Ensure reviews are free from protected class references
- Provide constructive feedback with improvement paths
- Document performance issues contemporaneously
- Offer equal development opportunities

SELF-AUDIT CHECKLIST:
□ Does this review focus on job performance?
□ Would this language be used for all employees?
□ Are there specific examples to support ratings?
□ Is feedback actionable and development-focused?
□ Are there any stereotypes or assumptions?
"""

# Add these to your existing knowledge base
new_documents = [age_bias_doc, coded_language_doc, debiasing_best_practices]

# If you have your existing chunks, extend them
# If starting fresh, create new chunks
all_documents = []

# Add your original 3 documents
try:
    for filename in ["eeoc_guidelines.txt", "debiasing_strategies.txt", "legal_cases.txt"]:
        with open(filename, "r") as f:
            all_documents.append(f.read())
    print("✅ Loaded original 3 documents")
except:
    print("⚠️  Original files not found - using new documents only")

# Add new documents
all_documents.extend(new_documents)

# Split into chunks (same function as before)
def split_text(text, chunk=500, overlap=50):
    parts = []
    start = 0
    while start < len(text):
        end = start + chunk
        parts.append(text[start:end])
        start += chunk - overlap
    return parts

all_chunks = []
for doc in all_documents:
    all_chunks.extend(split_text(doc))

print(f"✅ Total documents: {len(all_documents)}")
print(f"✅ Total chunks: {len(all_chunks)}")

# Rebuild FAISS index with expanded knowledge base
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(all_chunks)

# Build new index
expanded_index = faiss.IndexFlatL2(embeddings.shape[1])
expanded_index.add(np.array(embeddings))

print(f"✅ FAISS index rebuilt with {expanded_index.ntotal} vectors")
print("\n📚 Knowledge base now covers:")
print("   • Gender discrimination (EEOC Title VII)")
print("   • Age discrimination (ADEA)")
print("   • Coded language and implicit bias")
print("   • Appearance-based discrimination")
print("   • Vague language patterns")

EXPANDING RAG KNOWLEDGE BASE
✅ Loaded original 3 documents
✅ Total documents: 6
✅ Total chunks: 14
✅ FAISS index rebuilt with 14 vectors

📚 Knowledge base now covers:
   • Gender discrimination (EEOC Title VII)
   • Age discrimination (ADEA)
   • Coded language and implicit bias
   • Appearance-based discrimination
   • Vague language patterns


In [39]:
# Cell 39: Context-Aware RAG Retrieval

def query_rag_by_bias_type(bias_results, index, chunks, model, top_k=5):
    """
    Smart RAG: Query based on detected bias types
    Returns relevant legal context for specific bias dimensions
    """

    queries = []

    # Build queries based on detected bias types
    if 'gender_bias' in bias_results['bias_types_detected']:
        direction = bias_results['gender_bias']['direction']
        if direction == 'communal':
            queries.append("EEOC gender discrimination communal language stereotypes")
        elif direction == 'agentic':
            queries.append("EEOC gender discrimination agentic language bias")
        queries.append("Price Waterhouse v Hopkins gender stereotyping")

    if 'age_bias' in bias_results['bias_types_detected']:
        queries.append("Age Discrimination in Employment Act ADEA requirements")
        queries.append("age-related language performance reviews legal")

    if 'coded_language' in bias_results['bias_types_detected']:
        queries.append("implicit bias coded language employment discrimination")
        queries.append("race ethnicity coded language performance reviews")

    if 'appearance_bias' in bias_results['bias_types_detected']:
        queries.append("appearance grooming standards employment law")

    if 'vagueness' in bias_results['bias_types_detected']:
        queries.append("vague language performance reviews legal requirements")
        queries.append("objective criteria documentation employment")

    # If no specific bias detected, use general query
    if not queries:
        queries.append("fair performance review best practices EEOC")

    print(f"\n🔍 RAG QUERIES ({len(queries)} queries based on detected bias):")
    for i, q in enumerate(queries, 1):
        print(f"   {i}. {q}")

    # Retrieve documents for all queries
    all_retrieved = []
    seen_chunks = set()

    for query in queries:
        # Embed query
        query_embedding = model.encode([query])[0]

        # Search FAISS
        distances, indices = index.search(
            query_embedding.reshape(1, -1),
            k=top_k
        )

        # Collect unique chunks
        for idx in indices[0]:
            if idx < len(chunks):
                chunk_text = chunks[idx]
                chunk_key = chunk_text[:100]  # Use first 100 chars as key

                if chunk_key not in seen_chunks:
                    all_retrieved.append({
                        'text': chunk_text,
                        'query': query,
                        'distance': float(distances[0][list(indices[0]).index(idx)])
                    })
                    seen_chunks.add(chunk_key)

    # Sort by relevance (lower distance = more relevant)
    all_retrieved.sort(key=lambda x: x['distance'])

    # Return top documents
    top_docs = all_retrieved[:top_k]

    print(f"\n✅ Retrieved {len(top_docs)} relevant legal documents")

    return top_docs

# Test the context-aware RAG
print("="*70)
print("TESTING CONTEXT-AWARE RAG QUERY")
print("="*70)

# Use Martha Barnes example (has gender bias + vagueness)
test_file = 'sample_reviews/review_martha_barnes.pdf'
doc_processor = DocumentProcessor()
enhanced_detector = EnhancedBiasDetector()

extracted = doc_processor.process_document(test_file)
bias_results = enhanced_detector.detect_all_bias_dimensions(extracted['text'])

# Query RAG based on detected bias
legal_context = query_rag_by_bias_type(
    bias_results,
    expanded_index,
    all_chunks,
    model,
    top_k=5
)

print("\n📄 RETRIEVED LEGAL CONTEXT:")
for i, doc in enumerate(legal_context, 1):
    print(f"\n[Document {i}] (Distance: {doc['distance']:.3f})")
    print(f"Query: {doc['query']}")
    print(f"Content preview: {doc['text'][:200]}...")

TESTING CONTEXT-AWARE RAG QUERY

🔍 RAG QUERIES (4 queries based on detected bias):
   1. EEOC gender discrimination communal language stereotypes
   2. Price Waterhouse v Hopkins gender stereotyping
   3. implicit bias coded language employment discrimination
   4. race ethnicity coded language performance reviews

✅ Retrieved 5 relevant legal documents

📄 RETRIEVED LEGAL CONTEXT:

[Document 1] (Distance: 1.071)
Query: EEOC gender discrimination communal language stereotypes
Content preview:  Discrimination
Employers must ensure performance evaluation criteria are applied
consistently regardless of sex. Use of gender-stereotyped language
(e.g., describing women as "supportive" vs men as "...

[Document 2] (Distance: 1.110)
Query: EEOC gender discrimination communal language stereotypes
Content preview: 
EEOC Guidelines on Performance Evaluations

Section 202.6 - Adverse Impact Standard
The Four-Fifths Rule (80% Rule): A selection rate for any protected group
that is less than 80% (4/5t

In [52]:
# Cell 40: Enhanced FairnessAgent V2 - Complete Integration

class EnhancedFairnessAgentV2:
    """
    V2: Integrates new capabilities with original agent workflow

    New Capabilities:
    - Multi-format document processing (PDF, DOCX, TXT)
    - 5-dimensional bias detection
    - Context-aware RAG retrieval
    - Production audit logging
    - Iterative rewrite with self-verification
    """

    def __init__(self, api_key=None):
        # Initialize new components
        self.doc_processor = DocumentProcessor()
        self.enhanced_detector = EnhancedBiasDetector()
        self.logger = None  # Created per session

        # RAG components
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.index = expanded_index
        self.chunks = all_chunks

        # LLM setup
        if api_key:
            self.client = Anthropic(api_key=api_key)
        else:
            # Try to get from Colab secrets
            try:
                from google.colab import userdata
                api_key = userdata.get("ANTHROPIC_API_KEY")
                self.client = Anthropic(api_key=api_key)
            except:
                print("⚠️  No API key found - LLM rewrite will be skipped")
                self.client = None

    def process_document_complete(self, file_path, max_rewrite_attempts=3):
        """
        Complete end-to-end pipeline

        Steps:
        1. Extract text from document
        2. Multi-dimensional bias detection
        3. Context-aware RAG query
        4. LLM rewrite with legal grounding
        5. Self-verification with retry logic
        6. Audit logging throughout
        """

        # Initialize logger for this session
        from pathlib import Path
        self.logger = AuditLogger(f'audit_{Path(file_path).stem}.log')

        print("\n" + "="*70)
        print("ENHANCED FAIRNESS AGENT V2 - COMPLETE PIPELINE")
        print("="*70)
        print(f"Document: {Path(file_path).name}")
        print(f"Session: {self.logger.session_id}\n")

        # ================================================================
        # STEP 1: DOCUMENT EXTRACTION
        # ================================================================
        print("[STEP 1] DOCUMENT PROCESSING")
        print("-"*70)

        try:
            extracted = self.doc_processor.process_document(file_path)
            self.logger.log_document_processing(
                file_name=extracted['file_name'],
                file_format=extracted['format'],
                success=True,
                word_count=extracted['word_count']
            )
            print(f"✅ Extracted {extracted['word_count']} words from {extracted['format']}")
            original_text = extracted['text']
        except Exception as e:
            self.logger.log_document_processing(
                file_name=Path(file_path).name,
                file_format=Path(file_path).suffix,
                success=False,
                error=str(e)
            )
            print(f"❌ Extraction failed: {e}")
            self.logger.log_session_end()
            return None

        # ================================================================
        # STEP 2: MULTI-DIMENSIONAL BIAS DETECTION
        # ================================================================
        print("\n[STEP 2] 5-DIMENSIONAL BIAS DETECTION")
        print("-"*70)

        bias_results = self.enhanced_detector.detect_all_bias_dimensions(original_text)
        self.logger.log_bias_detection(
            file_name=extracted['file_name'],
            bias_results=bias_results
        )

        print(f"Risk Level: {bias_results['risk_level'].upper()}")
        print(f"Bias Score: {bias_results['overall_bias_score']:.2f}")
        print(f"Dimensions: {len(bias_results['bias_types_detected'])}")

        if bias_results['bias_types_detected']:
            print(f"Types: {', '.join([bt.replace('_', ' ').title() for bt in bias_results['bias_types_detected']])}")

        # If no significant bias, skip rewrite
        if bias_results['overall_bias_score'] < 0.25:
            print("\n✅ LOW BIAS - No rewrite needed")
            self.logger.log_session_end()
            return {
                'file_name': extracted['file_name'],
                'original_text': original_text,
                'bias_results': bias_results,
                'rewrite_needed': False,
                'session_id': self.logger.session_id
            }

        # ================================================================
        # STEP 3: CONTEXT-AWARE RAG QUERY
        # ================================================================
        print("\n[STEP 3] RETRIEVING LEGAL CONTEXT")
        print("-"*70)

        legal_docs = query_rag_by_bias_type(
            bias_results,
            self.index,
            self.chunks,
            self.model,
            top_k=5
        )

        self.logger.log_rag_query(
            query=f"Context for: {', '.join(bias_results['bias_types_detected'])}",
            documents_retrieved=len(legal_docs),
            success=True
        )

        # Combine legal context
        legal_context = "\n\n".join([doc['text'] for doc in legal_docs])

        # ================================================================
        # STEP 4: LLM REWRITE WITH RETRY LOGIC
        # ================================================================
        print("\n[STEP 4] LLM REWRITE WITH SELF-VERIFICATION")
        print("-"*70)

        if not self.client:
            print("⚠️  Skipping LLM rewrite (no API key)")
            self.logger.log_session_end()
            return {
                'file_name': extracted['file_name'],
                'original_text': original_text,
                'bias_results': bias_results,
                'legal_context': legal_context,
                'rewrite_needed': True,
                'rewrite_skipped': 'No API key',
                'session_id': self.logger.session_id
            }

        # Iterative rewrite with self-verification
        attempt = 0
        best_rewrite = None
        best_score = bias_results['overall_bias_score']

        while attempt < max_rewrite_attempts:
            attempt += 1
            print(f"\n   Attempt {attempt}/{max_rewrite_attempts}...")

            # Build prompt
            prompt = self._build_rewrite_prompt(
                original_text,
                bias_results,
                legal_context,
                attempt,
                best_score if attempt > 1 else None
            )

            # Call LLM
            try:
                response = self.client.messages.create(
                    model="claude-sonnet-4-20250514",
                    max_tokens=1000,
                    messages=[{"role": "user", "content": prompt}]
                )
                rewritten_text = response.content[0].text

                # Verify improvement
                new_bias_results = self.enhanced_detector.detect_all_bias_dimensions(rewritten_text)
                new_score = new_bias_results['overall_bias_score']

                improvement = bias_results['overall_bias_score'] - new_score

                print(f"   Original score: {bias_results['overall_bias_score']:.2f}")
                print(f"   New score: {new_score:.2f}")
                print(f"   Improvement: {improvement:.2f}")

                # Update best if improved
                if new_score < best_score:
                    best_rewrite = rewritten_text
                    best_score = new_score

                # Success if below threshold
                if new_score <= 0.5:
                    print(f"\n   ✅ SUCCESS - Bias reduced to acceptable level")

                    self.logger.log_llm_rewrite(
                        file_name=extracted['file_name'],
                        original_bias_score=bias_results['overall_bias_score'],
                        new_bias_score=new_score,
                        success=True
                    )

                    self.logger.log_session_end()

                    return {
                        'file_name': extracted['file_name'],
                        'original_text': original_text,
                        'rewritten_text': best_rewrite,
                        'original_bias_score': bias_results['overall_bias_score'],
                        'final_bias_score': new_score,
                        'attempts': attempt,
                        'success': True,
                        'session_id': self.logger.session_id
                    }

            except Exception as e:
                print(f"   ❌ Error: {e}")
                continue

        # Failed to meet threshold
        print(f"\n   ⚠️  Could not reduce bias below 0.5 after {max_rewrite_attempts} attempts")
        print(f"   Best achieved: {best_score:.2f}")
        print(f"   🚨 FLAGGED FOR HUMAN REVIEW")

        self.logger.log_llm_rewrite(
            file_name=extracted['file_name'],
            original_bias_score=bias_results['overall_bias_score'],
            new_bias_score=best_score,
            success=False
        )

        self.logger.log_session_end()

        return {
            'file_name': extracted['file_name'],
            'original_text': original_text,
            'best_rewrite': best_rewrite,
            'original_bias_score': bias_results['overall_bias_score'],
            'best_score': best_score,
            'attempts': max_rewrite_attempts,
            'success': False,
            'requires_human_review': True,
            'session_id': self.logger.session_id
        }

    def _build_rewrite_prompt(self, text, bias_results, legal_context, attempt, previous_score=None):
        """Build prompt for LLM rewrite"""

        bias_types_str = ', '.join([bt.replace('_', ' ') for bt in bias_results['bias_types_detected']])

        prompt = f"""You are an expert HR consultant tasked with rewriting a performance review to eliminate bias while preserving factual content.

ORIGINAL REVIEW:
{text}

DETECTED BIAS:
- Types: {bias_types_str}
- Overall bias score: {bias_results['overall_bias_score']:.2f} (target: ≤0.5)

LEGAL GUIDANCE:
{legal_context[:1500]}

REQUIREMENTS:
1. Preserve all factual performance information
2. Remove biased language identified above
3. Use objective, job-related criteria
4. Replace vague statements with specific examples where possible
5. Ensure language is neutral and professional
"""

        if attempt > 1 and previous_score:
            prompt += f"""
PREVIOUS ATTEMPT:
- Achieved score: {previous_score:.2f}
- Still above target of 0.5
- Focus on remaining bias indicators

Please generate an improved version that further reduces bias.
"""

        prompt += "\nProvide ONLY the rewritten review text, no explanations."

        return prompt

print("✅ EnhancedFairnessAgentV2 loaded successfully!")
print("   Capabilities: Document processing + 5D detection + RAG + LLM + Logging")

✅ EnhancedFairnessAgentV2 loaded successfully!
   Capabilities: Document processing + 5D detection + RAG + LLM + Logging


In [55]:
# Cell 41: Test Enhanced Agent V2 - Complete End-to-End

# Initialize agent
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
enhanced_agent_v2 = EnhancedFairnessAgentV2()

print("\n" + "="*70)
print("TESTING COMPLETE ENHANCED AGENT V2")
print("="*70)

# Test on Martha Barnes (high bias)
print("\n🧪 TEST 1: High Bias Document")
result = enhanced_agent_v2.process_document_complete('sample_reviews/review_martha_barnes.pdf')

if result and result.get('success'):
    print("\n" + "="*70)
    print("RESULTS SUMMARY")
    print("="*70)
    print(f"Original Bias Score: {result['original_bias_score']:.2f}")
    print(f"Final Bias Score: {result['final_bias_score']:.2f}")
    print(f"Attempts: {result['attempts']}")
    print(f"\nORIGINAL:")
    print(result['original_text'][:300] + "...")
    print(f"\nREWRITTEN:")
    print(result['rewritten_text'][:300] + "...")


TESTING COMPLETE ENHANCED AGENT V2

🧪 TEST 1: High Bias Document

ENHANCED FAIRNESS AGENT V2 - COMPLETE PIPELINE
Document: review_martha_barnes.pdf
Session: f3a66f6e

[STEP 1] DOCUMENT PROCESSING
----------------------------------------------------------------------
✅ Extracted 103 words from .pdf

[STEP 2] 5-DIMENSIONAL BIAS DETECTION
----------------------------------------------------------------------
Risk Level: HIGH
Bias Score: 0.55
Dimensions: 2
Types: Gender Bias, Coded Language

[STEP 3] RETRIEVING LEGAL CONTEXT
----------------------------------------------------------------------

🔍 RAG QUERIES (4 queries based on detected bias):
   1. EEOC gender discrimination communal language stereotypes
   2. Price Waterhouse v Hopkins gender stereotyping
   3. implicit bias coded language employment discrimination
   4. race ethnicity coded language performance reviews

✅ Retrieved 5 relevant legal documents

[STEP 4] LLM REWRITE WITH SELF-VERIFICATION
--------------------------------

In [56]:
from anthropic import Anthropic
from google.colab import userdata

client = Anthropic(api_key=userdata.get("ANTHROPIC_API_KEY"))

response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=50,
    messages=[{"role": "user", "content": "Say 'API works!'"}]
)

print("✅ API STATUS:", response.content[0].text)

✅ API STATUS: API works!


In [57]:
# Create additional sample PDFs for comprehensive demo

# Sample 4: Age Bias Example
create_sample_review_pdf(
    "review_age_discrimination.pdf",
    "Robert Chen - Senior Software Engineer",
    """Robert is a seasoned veteran with over 20 years of experience in traditional software development.
    While his expertise is valuable, he has been slow to adapt to modern agile methodologies and
    cloud-native technologies. As a longtime employee, Robert tends to be set in his established ways
    and shows some resistance to change. His old-school approach to problem-solving works but may not
    align with our digital transformation initiatives. We have concerns about his ability to keep pace
    with younger team members on emerging technologies. Recommendation: Consider retirement planning
    discussions. Training on modern DevOps practices may help. Rating: Meets Expectations."""
)

# Sample 5: Coded Language + Vagueness Example
create_sample_review_pdf(
    "review_coded_vague.pdf",
    "Maria Rodriguez - Sales Manager",
    """Maria is very articulate and well-spoken in client presentations, which is noteworthy.
    However, there are some concerns about cultural fit with the established team. While her sales
    numbers meet targets, something about her communication style just feels off. The team has
    raised questions about her approach, though it's hard to put a finger on exactly what the issue is.
    There's a gut feeling that she doesn't quite mesh with our team chemistry. Her different style
    and methods don't align well with how we've always done things here. Overall assessment is that
    she's not a good fit for this particular role despite adequate performance metrics.
    Rating: Below Expectations."""
)

print("\n" + "="*70)

✅ Created: sample_reviews/review_age_discrimination.pdf
✅ Created: sample_reviews/review_coded_vague.pdf



In [58]:
# Test Enhanced Agent V2 on all sample files

test_files = [
    ('Gender Bias (Communal)', 'sample_reviews/review_martha_barnes.pdf'),
    ('Gender Bias (Agentic)', 'sample_reviews/review_james_wilson.pdf'),
    ('Balanced/Neutral', 'sample_reviews/review_alex_chen.pdf'),
    ('Age Discrimination', 'sample_reviews/review_age_discrimination.pdf'),
    ('Coded + Vague', 'sample_reviews/review_coded_vague.pdf')
]

results_summary = []

print("="*70)
print("COMPREHENSIVE TESTING - ALL BIAS TYPES")
print("="*70)

for label, filepath in test_files:
    print(f"\n{'='*70}")
    print(f"TEST: {label}")
    print(f"{'='*70}")

    try:
        result = enhanced_agent_v2.process_document_complete(filepath, max_rewrite_attempts=2)

        if result:
            results_summary.append({
                'Type': label,
                'File': Path(filepath).name,
                'Original Score': f"{result.get('original_bias_score', 0):.2f}",
                'Final Score': f"{result.get('final_bias_score', result.get('best_score', 0)):.2f}",
                'Success': '✅' if result.get('success', False) else '⚠️'
            })
    except Exception as e:
        print(f"❌ Error: {e}")

    print("\n" + "-"*70)
    input("Press Enter for next test...")

# Print summary
print("\n" + "="*70)
print("BATCH TESTING SUMMARY")
print("="*70)

import pandas as pd
df = pd.DataFrame(results_summary)
print(df.to_string(index=False))


COMPREHENSIVE TESTING - ALL BIAS TYPES

TEST: Gender Bias (Communal)

ENHANCED FAIRNESS AGENT V2 - COMPLETE PIPELINE
Document: review_martha_barnes.pdf
Session: 2330a4c8

[STEP 1] DOCUMENT PROCESSING
----------------------------------------------------------------------
✅ Extracted 103 words from .pdf

[STEP 2] 5-DIMENSIONAL BIAS DETECTION
----------------------------------------------------------------------
Risk Level: HIGH
Bias Score: 0.55
Dimensions: 2
Types: Gender Bias, Coded Language

[STEP 3] RETRIEVING LEGAL CONTEXT
----------------------------------------------------------------------

🔍 RAG QUERIES (4 queries based on detected bias):
   1. EEOC gender discrimination communal language stereotypes
   2. Price Waterhouse v Hopkins gender stereotyping
   3. implicit bias coded language employment discrimination
   4. race ethnicity coded language performance reviews

✅ Retrieved 5 relevant legal documents

[STEP 4] LLM REWRITE WITH SELF-VERIFICATION
-----------------------------

In [59]:
# Test Enhanced Agent V2 on all sample files

test_files = [
    ('Gender Bias (Communal)', 'sample_reviews/review_martha_barnes.pdf'),
    ('Gender Bias (Agentic)', 'sample_reviews/review_james_wilson.pdf'),
    ('Balanced/Neutral', 'sample_reviews/review_alex_chen.pdf'),
    ('Age Discrimination', 'sample_reviews/review_age_discrimination.pdf'),
    ('Coded + Vague', 'sample_reviews/review_coded_vague.pdf')
]

results_summary = []

print("="*70)
print("COMPREHENSIVE TESTING - ALL BIAS TYPES")
print("="*70)

for label, filepath in test_files:
    print(f"\n{'='*70}")
    print(f"TEST: {label}")
    print(f"{'='*70}")

    try:
        result = enhanced_agent_v2.process_document_complete(filepath, max_rewrite_attempts=2)

        if result:
            results_summary.append({
                'Type': label,
                'File': Path(filepath).name,
                'Original Score': f"{result.get('original_bias_score', 0):.2f}",
                'Final Score': f"{result.get('final_bias_score', result.get('best_score', 0)):.2f}",
                'Success': '✅' if result.get('success', False) else '⚠️'
            })
    except Exception as e:
        print(f"❌ Error: {e}")

    print("\n" + "-"*70)
    input("Press Enter for next test...")

# Print summary
print("\n" + "="*70)
print("BATCH TESTING SUMMARY")
print("="*70)

import pandas as pd
df = pd.DataFrame(results_summary)
print(df.to_string(index=False))


COMPREHENSIVE TESTING - ALL BIAS TYPES

TEST: Gender Bias (Communal)

ENHANCED FAIRNESS AGENT V2 - COMPLETE PIPELINE
Document: review_martha_barnes.pdf
Session: 9c9c1fdc

[STEP 1] DOCUMENT PROCESSING
----------------------------------------------------------------------
✅ Extracted 103 words from .pdf

[STEP 2] 5-DIMENSIONAL BIAS DETECTION
----------------------------------------------------------------------
Risk Level: HIGH
Bias Score: 0.55
Dimensions: 2
Types: Gender Bias, Coded Language

[STEP 3] RETRIEVING LEGAL CONTEXT
----------------------------------------------------------------------

🔍 RAG QUERIES (4 queries based on detected bias):
   1. EEOC gender discrimination communal language stereotypes
   2. Price Waterhouse v Hopkins gender stereotyping
   3. implicit bias coded language employment discrimination
   4. race ethnicity coded language performance reviews

✅ Retrieved 5 relevant legal documents

[STEP 4] LLM REWRITE WITH SELF-VERIFICATION
-----------------------------